In [1]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
import polars as pl

from sklearn.model_selection import StratifiedGroupKFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import VotingClassifier

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.pipeline import Pipeline

import lightgbm as lgb
import catboost as cb
import xgboost as xgb

import optuna

In [2]:
root = Path('/kaggle/input/isic-2024-challenge')

train_path = root / 'train-metadata.csv'
test_path = root / 'test-metadata.csv'
subm_path = root / 'sample_submission.csv'

id_col = 'isic_id'
target_col = 'target'
group_col = 'patient_id'

err = 1e-5
sampling_ratio = 0.01
seed = 42

num_cols = [
    'age_approx',                        # Approximate age of patient at time of imaging.
    'clin_size_long_diam_mm',            # Maximum diameter of the lesion (mm).+
    'tbp_lv_A',                          # A inside  lesion.+
    'tbp_lv_Aext',                       # A outside lesion.+
    'tbp_lv_B',                          # B inside  lesion.+
    'tbp_lv_Bext',                       # B outside lesion.+ 
    'tbp_lv_C',                          # Chroma inside  lesion.+
    'tbp_lv_Cext',                       # Chroma outside lesion.+
    'tbp_lv_H',                          # Hue inside the lesion; calculated as the angle of A* and B* in LAB* color space. Typical values range from 25 (red) to 75 (brown).+
    'tbp_lv_Hext',                       # Hue outside lesion.+
    'tbp_lv_L',                          # L inside lesion.+
    'tbp_lv_Lext',                       # L outside lesion.+
    'tbp_lv_areaMM2',                    # Area of lesion (mm^2).+
    'tbp_lv_area_perim_ratio',           # Border jaggedness, the ratio between lesions perimeter and area. Circular lesions will have low values; irregular shaped lesions will have higher values. Values range 0-10.+
    'tbp_lv_color_std_mean',             # Color irregularity, calculated as the variance of colors within the lesion's boundary.
    'tbp_lv_deltaA',                     # Average A contrast (inside vs. outside lesion).+
    'tbp_lv_deltaB',                     # Average B contrast (inside vs. outside lesion).+
    'tbp_lv_deltaL',                     # Average L contrast (inside vs. outside lesion).+
    'tbp_lv_deltaLB',                    #
    'tbp_lv_deltaLBnorm',                # Contrast between the lesion and its immediate surrounding skin. Low contrast lesions tend to be faintly visible such as freckles; high contrast lesions tend to be those with darker pigment. Calculated as the average delta LB of the lesion relative to its immediate background in LAB* color space. Typical values range from 5.5 to 25.+
    'tbp_lv_eccentricity',               # Eccentricity.+
    'tbp_lv_minorAxisMM',                # Smallest lesion diameter (mm).+
    'tbp_lv_nevi_confidence',            # Nevus confidence score (0-100 scale) is a convolutional neural network classifier estimated probability that the lesion is a nevus. The neural network was trained on approximately 57,000 lesions that were classified and labeled by a dermatologist.+,++
    'tbp_lv_norm_border',                # Border irregularity (0-10 scale); the normalized average of border jaggedness and asymmetry.+
    'tbp_lv_norm_color',                 # Color variation (0-10 scale); the normalized average of color asymmetry and color irregularity.+
    'tbp_lv_perimeterMM',                # Perimeter of lesion (mm).+
    'tbp_lv_radial_color_std_max',       # Color asymmetry, a measure of asymmetry of the spatial distribution of color within the lesion. This score is calculated by looking at the average standard deviation in LAB* color space within concentric rings originating from the lesion center. Values range 0-10.+
    'tbp_lv_stdL',                       # Standard deviation of L inside  lesion.+
    'tbp_lv_stdLExt',                    # Standard deviation of L outside lesion.+
    'tbp_lv_symm_2axis',                 # Border asymmetry; a measure of asymmetry of the lesion's contour about an axis perpendicular to the lesion's most symmetric axis. Lesions with two axes of symmetry will therefore have low scores (more symmetric), while lesions with only one or zero axes of symmetry will have higher scores (less symmetric). This score is calculated by comparing opposite halves of the lesion contour over many degrees of rotation. The angle where the halves are most similar identifies the principal axis of symmetry, while the second axis of symmetry is perpendicular to the principal axis. Border asymmetry is reported as the asymmetry value about this second axis. Values range 0-10.+
    'tbp_lv_symm_2axis_angle',           # Lesion border asymmetry angle.+
    'tbp_lv_x',                          # X-coordinate of the lesion on 3D TBP.+
    'tbp_lv_y',                          # Y-coordinate of the lesion on 3D TBP.+
    'tbp_lv_z',                          # Z-coordinate of the lesion on 3D TBP.+
    
]

new_num_cols = [
    'x1',      # tbp_lv_minorAxisMM      / clin_size_long_diam_mm
    'x2',      # tbp_lv_areaMM2          / tbp_lv_perimeterMM **2
    'x3',      # tbp_lv_H                - tbp_lv_Hext              abs
    'x4',      # tbp_lv_L                - tbp_lv_Lext              abs
    'x5',      # tbp_lv_deltaA **2       + tbp_lv_deltaB **2 + tbp_lv_deltaL **2  sqrt  
    'x6',      # tbp_lv_norm_border      + tbp_lv_symm_2axis    
    'x7',      # tbp_lv_color_std_mean   / tbp_lv_radial_color_std_max
    'x8',      # tbp_lv_x **2 + tbp_lv_y **2 + tbp_lv_z **2  sqrt
    'x9',      # tbp_lv_perimeterMM      / tbp_lv_areaMM2
    'x10',     # tbp_lv_areaMM2          / tbp_lv_perimeterMM
    'x11',     # tbp_lv_deltaLBnorm      + tbp_lv_norm_color
    'x12',     # tbp_lv_symm_2axis       * tbp_lv_norm_border
    'x13',     # tbp_lv_symm_2axis       * tbp_lv_norm_border / (tbp_lv_symm_2axis + tbp_lv_norm_border)
    'x14',     # tbp_lv_stdL             / tbp_lv_Lext
    'x15',     # tbp_lv_stdL*tbp_lv_Lext / tbp_lv_stdL + tbp_lv_Lext
    'x16',     # clin_size_long_diam_mm  * age_approx
    'x17',     # tbp_lv_H                * tbp_lv_color_std_mean
    'x18',     # tbp_lv_norm_border      + tbp_lv_norm_color + tbp_lv_eccentricity / 3
    'x19',     # border_complexity       + lesion_shape_index
    'x20',     # tbp_lv_deltaA + tbp_lv_deltaB + tbp_lv_deltaL + tbp_lv_deltaLBnorm
    'x21',     # tbp_lv_areaMM2          + 1  np.log
    'x22',     # clin_size_long_diam_mm  / age_approx
    'x23',     # tbp_lv_H                + tbp_lv_Hext    / 2
    'x24',     # tbp_lv_deltaA **2 + tbp_lv_deltaB **2 + tbp_lv_deltaL **2   / 3  np.sqrt
    'x25',     # tbp_lv_color_std_mean   + bp_lv_area_perim_ratio + tbp_lv_symm_2axis   / 3
    'x26',     # tbp_lv_y                , tbp_lv_x  np.arctan2
    'x27',     # tbp_lv_deltaA           + tbp_lv_deltaB + tbp_lv_deltaL   / 3
    'x28',     # tbp_lv_symm_2axis       * tbp_lv_perimeterMM
    'x29',     # tbp_lv_area_perim_ratio + tbp_lv_eccentricity + bp_lv_norm_color + tbp_lv_symm_2axis   / 4
    'x30',     # tbp_lv_color_std_mean   / tbp_lv_stdLExt
    'x31',     # tbp_lv_norm_border      * tbp_lv_norm_color
    'x32',
    'x33',     # clin_size_long_diam_mm  / tbp_lv_deltaLBnorm
    'x34',     # tbp_lv_nevi_confidence  / age_approx
    'x35',
    'x36',     # tbp_lv_symm_2axis       * tbp_lv_radial_color_std_max
    'x37',     # tbp_lv_areaMM2          * sqrt(tbp_lv_x**2 + tbp_lv_y**2 + tbp_lv_z**2)
    'x38',     # abs(tbp_lv_L - tbp_lv_Lext) + abs(tbp_lv_A - tbp_lv_Aext) + abs(tbp_lv_B - tbp_lv_Bext)
    'x39',     # tbp_lv_eccentricity     * tbp_lv_color_std_mean
    'x40',     # tbp_lv_perimeterMM      / pi * sqrt(tbp_lv_areaMM2 / pi)
    'x41',     # age_approx              * clin_size_long_diam_mm * tbp_lv_symm_2axis
    'x42',     # age_approx              * tbp_lv_areaMM2 * tbp_lv_symm_2axis
]

new_num_cols2 = [    
    '_7_1',    # x1-x8   x2-x9   x3-x10  x4-x11  x5-x12  x6-x13  x7-x14
    '_7_2',    # x1-x15  x2-x16  x3-x17  x4-x18  x5-x19  x6-x20  x7-x21
    '_7_3',    # x1-x22  x2-x23  x3-x24  x4-x25  x5-x26  x6-x27  x7-x28
    '_7_4',    # x1-x29  x2-x30  x3-x31  x4-x32  x5-x33  x6-x34  x7-x35
    '_7_5',    # x1-x36  x2-x37  x3-x38  x4-x39  x5-x40  x6-x41  x7-x42
    '_7_6',    # x8-x15  x9-x16  x10-x17  x11-x18  x12-x19  x13-x20  x14-x21
    '_7_7',    # x8-x22  x9-x23  x10-x24  x11-x25  x12-x26  x13-x27  x14-x28
    '_7_8',    # x8-x29  x9-x30  x10-x31  x11-x32  x12-x33  x13-x34  x14-x35
    '_7_9',    # x8-x36  x9-x37  x10-x38  x11-x39  x12-x40  x13-x41  x14-x42
    '_7_10',   # x15-x22  x16-x23  x17-x24  x18-x25  x19-x26  x20-x27  x21-x28
    '_7_11',   # x15-x29  x16-x30  x17-x31  x18-x32  x19-x33  x20-x34  x21-x35
    '_7_12',   # x15-x36  x16-x37  x17-x38  x18-x39  x19-x40  x20-x41  x21-x42
    '_7_13',   # x22-x29  x23-x30  x24-x31  x25-x32  x26-x33  x27-x34  x28-x35
    '_7_14',   # x22-x36  x23-x37  x24-x38  x25-x39  x26-x40  x27-x41  x28-x42
    '_7_15',   # x29-x36  x30-x37  x31-x38  x32-x39  x33-x40  x34-x41  x35-x42
    
    '_6_1',    # x1-x7   x2-x8   x3-x9   x4-x10  x5-x11  x6-x12
    '_6_2',    # x1-x13  x2-x14  x3-x15  x4-x16  x5-x17  x6-x18
    '_6_3',    # x1-x19  x2-x20  x3-x21  x4-x22  x5-x23  x6-x24
    '_6_4',    # x1-x25  x2-x26  x3-x27  x4-x28  x5-x29  x6-x30
    '_6_5',    # x1-x31  x2-x32  x3-x33  x4-x34  x5-x35  x6-x36
    '_6_6',    # x1-x37  x2-x38  x3-x39  x4-x40  x5-x41  x6-x42
    '_6_7',    # x7-x13  x8-x14  x9-x15  x10-x16  x11-x17  x12-x23
    '_6_8',    # x7-x19  x8-x20  x9-x21  x10-x22  x11-x23  x12-x24
    '_6_9',    # x7-x25  x8-x26  x9-x27  x10-x28  x11-x29  x12-x30
    '_6_10',   # x7-x31  x8-x32  x9-x33  x10-x34  x11-x35  x12-x36
    '_6_11',   # x7-x37  x8-x38  x9-x39  x10-x40  x11-x41  x12-x42
    '_6_12',   # x13-x19  x14-x20  x15-x21  x16-x22  x17-x23  x18-x24
    '_6_13',   # x13-x25  x14-x26  x15-x27  x16-x28  x17-x29  x18-x30
    '_6_14',   # x13-x31  x14-x32  x15-x33  x16-x34  x17-x35  x18-x36
    '_6_15',   # x13-x37  x14-x38  x15-x39  x16-x40  x17-x41  x18-x42
    '_6_16',   # x19-x25  x20-x26  x21-x27  x22-x28  x23-x29  x24-x30
    '_6_17',   # x19-x31  x20-x32  x21-x33  x22-x34  x23-x35  x24-x36
    '_6_18',   # x19-x37  x20-x38  x21-x39  x22-x40  x23-x41  x24-x42
    '_6_19',   # x25-x31  x26-x32  x27-x33  x28-x34  x29-x35  x30-x36
    '_6_20',   # x25-x37  x26-x38  x27-x39  x28-x40  x29-x41  x30-x42
    '_6_21',   # x31-x37  x32-x38  x33-x39  x34-x40  x35-x41  x36-x42
    
    '_5_1',    # x1-x6    x2-x7    x3-x8    x4-x9    x5-x10
    '_5_2',    # x1-x11   x2-x12   x3-x13   x4-x14   x5-x15
    '_5_3',    # x1-x16   x2-x17   x3-x18   x4-x19   x5-x20
    '_5_4',    # x1-x21   x2-x22   x3-x23   x4-x24   x5-x25
    '_5_5',    # x1-x26   x2-x27   x3-x28   x4-x29   x5-x30
    '_5_6',    # x1-x31   x2-x32   x3-x33   x4-x34   x5-x35
    '_5_7',    # x1-x36   x2-x37   x3-x38   x4-x39   x5-x40
    '_5_8',    # x6-x11   x7-x12   x8-x13   x9-x14   x10-x15
    '_5_9',    # x6-x16   x7-x17   x8-x18   x9-x19   x10-x20
    '_5_10',   # x6-x21   x7-x22   x8-x23   x9-x24   x10-x25
    '_5_11',   # x6-x26   x7-x27   x8-x28   x9-x29   x10-x30
    '_5_12',   # x6-x31   x7-x32   x8-x33   x9-x34   x10-x35
    '_5_13',   # x6-x36   x7-x37   x8-x38   x9-x39   x10-x40
    '_5_14',   # x11-x16   x12-x17   x13-x18   x14-x19   x15-x20
    '_5_15',   # x11-x21   x12-x22   x13-x23   x14-x24   x15-x25
    '_5_16',   # x11-x26   x12-x27   x13-x28   x14-x29   x15-x30
    '_5_17',   # x11-x31   x12-x32   x13-x33   x14-x34   x15-x35
    '_5_18',   # x11-x36   x12-x37   x13-x38   x14-x39   x15-x40
    '_5_19',   # x16-x21   x17-x22   x18-x23   x19-x24   x20-x25
    '_5_20',   # x16-x26   x17-x27   x18-x28   x19-x29   x20-x30
    '_5_21',   # x16-x31   x17-x32   x18-x33   x19-x34   x20-x35
    '_5_22',   # x16-x36   x17-x37   x18-x38   x19-x39   x20-x40
    '_5_23',   # x21-x26   x22-x27   x23-x28   x24-x29   x25-x30
    '_5_24',   # x21-x31   x22-x32   x23-x33   x24-x34   x25-x35
    '_5_25',   # x21-x36   x22-x37   x23-x38   x24-x39   x25-x40
    '_5_26',   # x26-x31   x27-x32   x28-x33   x29-x34   x30-x35
    '_5_27',   # x26-x36   x27-x37   x28-x38   x29-x39   x30-x40
    '_5_28',   # x31-x36   x32-x37   x33-x38   x34-x39   x35-x40
    
    '_4_1',    # x1-x5    x2-x6    x3-x7    x4-x8    
    '_4_2',    # x1-x9    x2-x10   x3-x11   x4-x12
    '_4_3',    # x1-x13   x2-x14   x3-x15   x4-x16
    '_4_4',    # x1-x17   x2-x18   x3-x19   x4-x20
    '_4_5',    # x1-x21   x2-x22   x3-x23   x4-x24
    '_4_6',    # x1-x25   x2-x26   x3-x27   x4-x28
    '_4_7',    # x1-x29   x2-x30   x3-x31   x4-x32
    '_4_8',    # x1-x33   x2-x34   x3-x35   x4-x36
    '_4_9',    # x1-x37   x2-x38   x3-x39   x4-x40
    '_4_10',   # x5-x9    x6-x10   x7-x11   x8-x12
    '_4_11',   # x5-x13   x6-x14   x7-x15   x8-x16
    '_4_12',   # x5-x17   x6-x18   x7-x19   x8-x20
    '_4_13',   # x5-x21   x6-x22   x7-x23   x8-x24
    '_4_14',   # x5-x25   x6-x26   x7-x27   x8-x28
    '_4_15',   # x5-x29   x6-x30   x7-x31   x8-x32
    '_4_16',   # x5-x33   x6-x34   x7-x35   x8-x36
    '_4_17',   # x5-x37   x6-x38   x7-x39   x8-x40
    '_4_18',   # x9-x13   x10-x14   x11-x15   x12-x16
    '_4_19',   # x9-x17   x10-x18   x11-x19   x12-x20
    '_4_20',   # x9-x21   x10-x22   x11-x23   x12-x24
    '_4_21',   # x9-x25   x10-x26   x11-x27   x12-x28
    '_4_22',   # x9-x29   x10-x30   x11-x31   x12-x32
    '_4_23',   # x9-x33   x10-x34   x11-x35   x12-x36
    '_4_24',   # x9-x37   x10-x38   x11-x39   x12-x40
    '_4_25',   # x13-x17   x14-x18   x15-x19   x16-x20
    '_4_26',   # x13-x21   x14-x22   x15-x23   x16-x24
    '_4_27',   # x13-x25   x14-x26   x15-x27   x16-x28
    '_4_28',   # x13-x29   x14-x30   x15-x31   x16-x32
    '_4_29',   # x13-x33   x14-x34   x15-x35   x16-x36
    '_4_30',   # x13-x37   x14-x38   x15-x39   x16-x40
    '_4_31',   # x17-x21   x18-x22   x19-x23   x20-x24
    '_4_32',   # x17-x25   x18-x26   x19-x27   x20-x28
    '_4_33',   # x17-x29   x18-x30   x19-x31   x20-x32
    '_4_34',   # x17-x33   x18-x34   x19-x35   x20-x36
    '_4_35',   # x17-x37   x18-x38   x19-x39   x20-x40
    '_4_36',   # x21-x25   x22-x26   x23-x27   x24-x28
    '_4_37',   # x21-x29   x22-x30   x23-x31   x24-x32
    '_4_38',   # x21-x33   x22-x34   x23-x35   x24-x36
    '_4_39',   # x21-x37   x22-x38   x23-x39   x24-x40
    '_4_40',   # x25-x29   x26-x30   x27-x31   x28-x32
    '_4_41',   # x25-x33   x26-x34   x27-x35   x28-x36
    '_4_42',   # x25-x37   x26-x38   x27-x39   x28-x40
    '_4_43',   # x29-x33   x30-x34   x31-x35   x32-x36
    '_4_44',   # x29-x37   x30-x38   x31-x39   x32-x40
    '_4_45',   # x33-x37   x34-x38   x35-x39   x36-x40
]

cat_cols = ['sex', 'anatom_site_general', 'tbp_tile_type', 'tbp_lv_location', 'tbp_lv_location_simple', 'attribution']
norm_cols = [f'{col}_patient_norm' for col in num_cols + new_num_cols]
special_cols = ['count_per_patient']
feature_cols = num_cols + new_num_cols + new_num_cols2 + cat_cols + norm_cols + special_cols # 

In [3]:
def read_data(path):
    return (
        pl.read_csv(path)
        .with_columns(
            pl.col('age_approx').cast(pl.String).replace('NA', np.nan).cast(pl.Float64),
        )
        .with_columns(
            pl.col(pl.Float64).fill_nan(pl.col(pl.Float64).median()), # You may want to impute test data with train
        )
        .with_columns(
            x1    = pl.col('tbp_lv_minorAxisMM') / pl.col('clin_size_long_diam_mm'),
            x2    = pl.col('tbp_lv_areaMM2') / (pl.col('tbp_lv_perimeterMM') ** 2),
            x3    = (pl.col('tbp_lv_H') - pl.col('tbp_lv_Hext')).abs(),
            x4    = (pl.col('tbp_lv_L') - pl.col('tbp_lv_Lext')).abs(),
            x5    = (pl.col('tbp_lv_deltaA') ** 2 + pl.col('tbp_lv_deltaB') ** 2 + pl.col('tbp_lv_deltaL') ** 2).sqrt(),
            x6    = pl.col('tbp_lv_norm_border') + pl.col('tbp_lv_symm_2axis'),
            x7    = pl.col('tbp_lv_color_std_mean') / (pl.col('tbp_lv_radial_color_std_max') + err),
        )
        .with_columns(
            x8    = (pl.col('tbp_lv_x') ** 2 + pl.col('tbp_lv_y') ** 2 + pl.col('tbp_lv_z') ** 2).sqrt(),
            x9    = pl.col('tbp_lv_perimeterMM') / pl.col('tbp_lv_areaMM2'),
            x10   = pl.col('tbp_lv_areaMM2') / pl.col('tbp_lv_perimeterMM'),
            x11   = pl.col('tbp_lv_deltaLBnorm') + pl.col('tbp_lv_norm_color'),
            xc1   = pl.col('anatom_site_general') + '_' + pl.col('tbp_lv_location'),
            x12   = pl.col('tbp_lv_symm_2axis') * pl.col('tbp_lv_norm_border'),
            x13   = pl.col('tbp_lv_symm_2axis') * pl.col('tbp_lv_norm_border') / (pl.col('tbp_lv_symm_2axis') + pl.col('tbp_lv_norm_border')),
        )
        .with_columns(
            x14   = pl.col('tbp_lv_stdL') / pl.col('tbp_lv_Lext'),
            x15   = pl.col('tbp_lv_stdL') * pl.col('tbp_lv_Lext') / (pl.col('tbp_lv_stdL') + pl.col('tbp_lv_Lext')),
            x16   = pl.col('clin_size_long_diam_mm') * pl.col('age_approx'),
            x17   = pl.col('tbp_lv_H') * pl.col('tbp_lv_color_std_mean'),
            x18   = (pl.col('tbp_lv_norm_border') + pl.col('tbp_lv_norm_color') + pl.col('tbp_lv_eccentricity')) / 3,
            x19   = pl.col('x6') + pl.col('x2'),
            x20   = pl.col('tbp_lv_deltaA') + pl.col('tbp_lv_deltaB') + pl.col('tbp_lv_deltaL') + pl.col('tbp_lv_deltaLBnorm'),
        )
        .with_columns(
            x21   = (pl.col('tbp_lv_areaMM2') + 1).log(),
            x22   = pl.col('clin_size_long_diam_mm') / pl.col('age_approx'),
            x23   = (pl.col('tbp_lv_H') + pl.col('tbp_lv_Hext')) / 2,
            x24   = ((pl.col('tbp_lv_deltaA') ** 2 + pl.col('tbp_lv_deltaB') ** 2 + pl.col('tbp_lv_deltaL') ** 2) / 3).sqrt(),
            x25   = (pl.col('tbp_lv_color_std_mean') + pl.col('tbp_lv_area_perim_ratio') + pl.col('tbp_lv_symm_2axis')) / 3,
            x26   = pl.arctan2(pl.col('tbp_lv_y'), pl.col('tbp_lv_x')),
            x27   = (pl.col('tbp_lv_deltaA') + pl.col('tbp_lv_deltaB') + pl.col('tbp_lv_deltaL')) / 3,
        )
        .with_columns(
            x28   = pl.col('tbp_lv_symm_2axis') * pl.col('tbp_lv_perimeterMM'),
            x29   = (pl.col('tbp_lv_area_perim_ratio') + pl.col('tbp_lv_eccentricity') + pl.col('tbp_lv_norm_color') + pl.col('tbp_lv_symm_2axis')) / 4,
            x30   = pl.col('tbp_lv_color_std_mean') / pl.col('tbp_lv_stdLExt'),
            x31   = pl.col('tbp_lv_norm_border') * pl.col('tbp_lv_norm_color'),
            x32   = pl.col('tbp_lv_norm_border') * pl.col('tbp_lv_norm_color') / (pl.col('tbp_lv_norm_border') + pl.col('tbp_lv_norm_color')),
            x33   = pl.col('clin_size_long_diam_mm') / pl.col('tbp_lv_deltaLBnorm'),
            x34   = pl.col('tbp_lv_nevi_confidence') / pl.col('age_approx'),
            x35   = (pl.col('clin_size_long_diam_mm')**2 + pl.col('age_approx')**2).sqrt(),
            x36   = pl.col('tbp_lv_radial_color_std_max') * pl.col('tbp_lv_symm_2axis'),
        )
        .with_columns(
            x37   = pl.col('tbp_lv_areaMM2') * (pl.col('tbp_lv_x')**2 + pl.col('tbp_lv_y')**2 + pl.col('tbp_lv_z')**2).sqrt(),
            x38   = (pl.col('tbp_lv_L') - pl.col('tbp_lv_Lext')).abs() + (pl.col('tbp_lv_A') - pl.col('tbp_lv_Aext')).abs() + (pl.col('tbp_lv_B') - pl.col('tbp_lv_Bext')).abs(),
            x39   = pl.col('tbp_lv_eccentricity') * pl.col('tbp_lv_color_std_mean'),
            x40   = pl.col('tbp_lv_perimeterMM') / (2 * np.pi * (pl.col('tbp_lv_areaMM2') / np.pi).sqrt()),
            x41   = pl.col('age_approx') * pl.col('clin_size_long_diam_mm') * pl.col('tbp_lv_symm_2axis'),
            x42   = pl.col('age_approx') * pl.col('tbp_lv_areaMM2') * pl.col('tbp_lv_symm_2axis'),
        )
        .with_columns(
            _7_1  = ((pl.col('x1')-pl.col('x8'))**2 +(pl.col('x2')-pl.col('x9'))**2 +(pl.col('x3')-pl.col('x10'))**2+(pl.col('x4')-pl.col('x11'))**2+(pl.col('x5')-pl.col('x12'))**2+(pl.col('x6')-pl.col('x13'))**2+(pl.col('x7')-pl.col('x14'))**2).sqrt(),
            _7_2  = ((pl.col('x1')-pl.col('x15'))**2+(pl.col('x2')-pl.col('x16'))**2+(pl.col('x3')-pl.col('x17'))**2+(pl.col('x4')-pl.col('x18'))**2+(pl.col('x5')-pl.col('x19'))**2+(pl.col('x6')-pl.col('x20'))**2+(pl.col('x7')-pl.col('x21'))**2).sqrt(),
            _7_3  = ((pl.col('x1')-pl.col('x22'))**2+(pl.col('x2')-pl.col('x23'))**2+(pl.col('x3')-pl.col('x24'))**2+(pl.col('x4')-pl.col('x25'))**2+(pl.col('x5')-pl.col('x26'))**2+(pl.col('x6')-pl.col('x27'))**2+(pl.col('x7')-pl.col('x28'))**2).sqrt(),
            _7_4  = ((pl.col('x1')-pl.col('x29'))**2+(pl.col('x2')-pl.col('x30'))**2+(pl.col('x3')-pl.col('x31'))**2+(pl.col('x4')-pl.col('x32'))**2+(pl.col('x5')-pl.col('x33'))**2+(pl.col('x6')-pl.col('x34'))**2+(pl.col('x7')-pl.col('x35'))**2).sqrt(),
            _7_5  = ((pl.col('x1')-pl.col('x36'))**2+(pl.col('x2')-pl.col('x37'))**2+(pl.col('x3')-pl.col('x38'))**2+(pl.col('x4')-pl.col('x39'))**2+(pl.col('x5')-pl.col('x40'))**2+(pl.col('x6')-pl.col('x41'))**2+(pl.col('x7')-pl.col('x42'))**2).sqrt(),
            _7_6  = ((pl.col('x8')-pl.col('x15'))**2+(pl.col('x9')-pl.col('x16'))**2+(pl.col('x10')-pl.col('x17'))**2+(pl.col('x11')-pl.col('x18'))**2+(pl.col('x12')-pl.col('x19'))**2+(pl.col('x13')-pl.col('x20'))**2+(pl.col('x14')-pl.col('x21'))**2).sqrt(),
            _7_7  = ((pl.col('x8')-pl.col('x22'))**2+(pl.col('x9')-pl.col('x23'))**2+(pl.col('x10')-pl.col('x24'))**2+(pl.col('x11')-pl.col('x25'))**2+(pl.col('x12')-pl.col('x26'))**2+(pl.col('x13')-pl.col('x27'))**2+(pl.col('x14')-pl.col('x28'))**2).sqrt(),
            _7_8  = ((pl.col('x8')-pl.col('x29'))**2+(pl.col('x9')-pl.col('x30'))**2+(pl.col('x10')-pl.col('x31'))**2+(pl.col('x11')-pl.col('x32'))**2+(pl.col('x12')-pl.col('x33'))**2+(pl.col('x13')-pl.col('x34'))**2+(pl.col('x14')-pl.col('x35'))**2).sqrt(),
            _7_9  = ((pl.col('x8')-pl.col('x36'))**2+(pl.col('x9')-pl.col('x37'))**2+(pl.col('x10')-pl.col('x38'))**2+(pl.col('x11')-pl.col('x39'))**2+(pl.col('x12')-pl.col('x40'))**2+(pl.col('x13')-pl.col('x41'))**2+(pl.col('x14')-pl.col('x42'))**2).sqrt(),
            _7_10 = ((pl.col('x15')-pl.col('x22'))**2+(pl.col('x16')-pl.col('x23'))**2+(pl.col('x17')-pl.col('x24'))**2+(pl.col('x18')-pl.col('x25'))**2+(pl.col('x19')-pl.col('x26'))**2+(pl.col('x20')-pl.col('x27'))**2+(pl.col('x21')-pl.col('x28'))**2).sqrt(),
            _7_11 = ((pl.col('x15')-pl.col('x29'))**2+(pl.col('x16')-pl.col('x30'))**2+(pl.col('x17')-pl.col('x31'))**2+(pl.col('x18')-pl.col('x32'))**2+(pl.col('x19')-pl.col('x33'))**2+(pl.col('x20')-pl.col('x34'))**2+(pl.col('x21')-pl.col('x35'))**2).sqrt(),
            _7_12 = ((pl.col('x15')-pl.col('x36'))**2+(pl.col('x16')-pl.col('x37'))**2+(pl.col('x17')-pl.col('x38'))**2+(pl.col('x18')-pl.col('x39'))**2+(pl.col('x19')-pl.col('x40'))**2+(pl.col('x20')-pl.col('x41'))**2+(pl.col('x21')-pl.col('x42'))**2).sqrt(),
            _7_13 = ((pl.col('x22')-pl.col('x29'))**2+(pl.col('x23')-pl.col('x30'))**2+(pl.col('x24')-pl.col('x31'))**2+(pl.col('x25')-pl.col('x32'))**2+(pl.col('x26')-pl.col('x33'))**2+(pl.col('x27')-pl.col('x34'))**2+(pl.col('x28')-pl.col('x35'))**2).sqrt(),
            _7_14 = ((pl.col('x22')-pl.col('x36'))**2+(pl.col('x23')-pl.col('x37'))**2+(pl.col('x24')-pl.col('x38'))**2+(pl.col('x25')-pl.col('x39'))**2+(pl.col('x26')-pl.col('x40'))**2+(pl.col('x27')-pl.col('x41'))**2+(pl.col('x28')-pl.col('x42'))**2).sqrt(),
            _7_15 = ((pl.col('x29')-pl.col('x36'))**2+(pl.col('x30')-pl.col('x37'))**2+(pl.col('x31')-pl.col('x38'))**2+(pl.col('x32')-pl.col('x39'))**2+(pl.col('x33')-pl.col('x40'))**2+(pl.col('x34')-pl.col('x41'))**2+(pl.col('x35')-pl.col('x42'))**2).sqrt(),
        )
                           #  = ((pl.col('    ')-pl.col('   '))**2+(pl.col('   '))**2).sqrt(),
        .with_columns(
            _6_1  = ((pl.col('x1')-pl.col('x7'))**2+(pl.col('x2')-pl.col('x8'))**2+(pl.col('x3')-pl.col('x9'))**2+(pl.col('x4')-pl.col('x10'))**2+(pl.col('x5')-pl.col('x11'))**2+(pl.col('x6')-pl.col('x12'))**2).sqrt(),
            _6_2  = ((pl.col('x1')-pl.col('x13'))**2+(pl.col('x2')-pl.col('x14'))**2+(pl.col('x3')-pl.col('x15'))**2+(pl.col('x4')-pl.col('x16'))**2+(pl.col('x5')-pl.col('x17'))**2+(pl.col('x6')-pl.col('x18'))**2).sqrt(),
            _6_3  = ((pl.col('x1')-pl.col('x19'))**2+(pl.col('x2')-pl.col('x20'))**2+(pl.col('x3')-pl.col('x21'))**2+(pl.col('x4')-pl.col('x22'))**2+(pl.col('x5')-pl.col('x23'))**2+(pl.col('x6')-pl.col('x24'))**2).sqrt(),
            _6_4  = ((pl.col('x1')-pl.col('x25'))**2+(pl.col('x2')-pl.col('x26'))**2+(pl.col('x3')-pl.col('x27'))**2+(pl.col('x4')-pl.col('x28'))**2+(pl.col('x5')-pl.col('x29'))**2+(pl.col('x6')-pl.col('x30'))**2).sqrt(),
            _6_5  = ((pl.col('x1')-pl.col('x31'))**2+(pl.col('x2')-pl.col('x32'))**2+(pl.col('x3')-pl.col('x33'))**2+(pl.col('x4')-pl.col('x34'))**2+(pl.col('x5')-pl.col('x35'))**2+(pl.col('x6')-pl.col('x36'))**2).sqrt(),
            _6_6  = ((pl.col('x1')-pl.col('x37'))**2+(pl.col('x2')-pl.col('x38'))**2+(pl.col('x3')-pl.col('x39'))**2+(pl.col('x4')-pl.col('x40'))**2+(pl.col('x5')-pl.col('x41'))**2+(pl.col('x6')-pl.col('x42'))**2).sqrt(),
            _6_7  = ((pl.col('x7')-pl.col('x13'))**2+(pl.col('x8')-pl.col('x14'))**2+(pl.col('x9')-pl.col('x15'))**2+(pl.col('x10')-pl.col('x16'))**2+(pl.col('x11')-pl.col('x17'))**2+(pl.col('x12')-pl.col('x23'))**2).sqrt(),
            _6_8  = ((pl.col('x7')-pl.col('x19'))**2+(pl.col('x8')-pl.col('x20'))**2+(pl.col('x9')-pl.col('x21'))**2+(pl.col('x10')-pl.col('x22'))**2+(pl.col('x11')-pl.col('x23'))**2+(pl.col('x12')-pl.col('x24'))**2).sqrt(),
            _6_9  = ((pl.col('x7')-pl.col('x25'))**2+(pl.col('x8')-pl.col('x26'))**2+(pl.col('x9')-pl.col('x27'))**2+(pl.col('x10')-pl.col('x28'))**2+(pl.col('x11')-pl.col('x29'))**2+(pl.col('x12')-pl.col('x30'))**2).sqrt(),
            _6_10 = ((pl.col('x7')-pl.col('x31'))**2+(pl.col('x8')-pl.col('x32'))**2+(pl.col('x9')-pl.col('x33'))**2+(pl.col('x10')-pl.col('x34'))**2+(pl.col('x11')-pl.col('x35'))**2+(pl.col('x12')-pl.col('x36'))**2).sqrt(),
            _6_11 = ((pl.col('x7')-pl.col('x37'))**2+(pl.col('x8')-pl.col('x38'))**2+(pl.col('x9')-pl.col('x39'))**2+(pl.col('x10')-pl.col('x40'))**2+(pl.col('x11')-pl.col('x41'))**2+(pl.col('x12')-pl.col('x42'))**2).sqrt(),
            _6_12 = ((pl.col('x13')-pl.col('x19'))**2+(pl.col('x14')-pl.col('x20'))**2+(pl.col('x15')-pl.col('x21'))**2+(pl.col('x16')-pl.col('x22'))**2+(pl.col('x17')-pl.col('x23'))**2+(pl.col('x18')-pl.col('x24'))**2).sqrt(),
            _6_13 = ((pl.col('x13')-pl.col('x25'))**2+(pl.col('x14')-pl.col('x26'))**2+(pl.col('x15')-pl.col('x27'))**2+(pl.col('x16')-pl.col('x28'))**2+(pl.col('x17')-pl.col('x29'))**2+(pl.col('x18')-pl.col('x30'))**2).sqrt(),
            _6_14 = ((pl.col('x13')-pl.col('x31'))**2+(pl.col('x14')-pl.col('x32'))**2+(pl.col('x15')-pl.col('x33'))**2+(pl.col('x16')-pl.col('x34'))**2+(pl.col('x17')-pl.col('x35'))**2+(pl.col('x18')-pl.col('x36'))**2).sqrt(),
            _6_15 = ((pl.col('x13')-pl.col('x37'))**2+(pl.col('x14')-pl.col('x38'))**2+(pl.col('x15')-pl.col('x39'))**2+(pl.col('x16')-pl.col('x40'))**2+(pl.col('x17')-pl.col('x41'))**2+(pl.col('x18')-pl.col('x42'))**2).sqrt(),
            _6_16 = ((pl.col('x19')-pl.col('x25'))**2+(pl.col('x20')-pl.col('x26'))**2+(pl.col('x21')-pl.col('x27'))**2+(pl.col('x22')-pl.col('x28'))**2+(pl.col('x23')-pl.col('x29'))**2+(pl.col('x24')-pl.col('x30'))**2).sqrt(),
            _6_17 = ((pl.col('x19')-pl.col('x31'))**2+(pl.col('x20')-pl.col('x32'))**2+(pl.col('x21')-pl.col('x33'))**2+(pl.col('x22')-pl.col('x34'))**2+(pl.col('x23')-pl.col('x35'))**2+(pl.col('x24')-pl.col('x36'))**2).sqrt(),
            _6_18 = ((pl.col('x19')-pl.col('x37'))**2+(pl.col('x20')-pl.col('x38'))**2+(pl.col('x21')-pl.col('x39'))**2+(pl.col('x22')-pl.col('x40'))**2+(pl.col('x23')-pl.col('x41'))**2+(pl.col('x24')-pl.col('x42'))**2).sqrt(),
            _6_19 = ((pl.col('x25')-pl.col('x31'))**2+(pl.col('x26')-pl.col('x32'))**2+(pl.col('x27')-pl.col('x33'))**2+(pl.col('x28')-pl.col('x34'))**2+(pl.col('x29')-pl.col('x35'))**2+(pl.col('x30')-pl.col('x36'))**2).sqrt(),
            _6_20 = ((pl.col('x25')-pl.col('x37'))**2+(pl.col('x26')-pl.col('x38'))**2+(pl.col('x27')-pl.col('x39'))**2+(pl.col('x28')-pl.col('x40'))**2+(pl.col('x29')-pl.col('x41'))**2+(pl.col('x30')-pl.col('x42'))**2).sqrt(),
            _6_21 = ((pl.col('x31')-pl.col('x37'))**2+(pl.col('x32')-pl.col('x38'))**2+(pl.col('x33')-pl.col('x39'))**2+(pl.col('x34')-pl.col('x40'))**2+(pl.col('x35')-pl.col('x41'))**2+(pl.col('x36')-pl.col('x42'))**2).sqrt(),
        )                    
# #                            #  = ((pl.col('    ')-pl.col('   '))**2+(pl.col('   '))**2).sqrt(),
        .with_columns(  
            _5_1  = ((pl.col('x1')-pl.col('x6'))**2 +(pl.col('x2')-pl.col('x7'))**2 +(pl.col('x3')-pl.col('x8'))**2 +(pl.col('x4')-pl.col('x9'))**2 +(pl.col('x5')-pl.col('x10'))**2).sqrt(),
            _5_2  = ((pl.col('x1')-pl.col('x11'))**2+(pl.col('x2')-pl.col('x12'))**2+(pl.col('x3')-pl.col('x13'))**2+(pl.col('x4')-pl.col('x14'))**2+(pl.col('x5')-pl.col('x15'))**2).sqrt(),
            _5_3  = ((pl.col('x1')-pl.col('x16'))**2+(pl.col('x2')-pl.col('x17'))**2+(pl.col('x3')-pl.col('x18'))**2+(pl.col('x4')-pl.col('x19'))**2+(pl.col('x5')-pl.col('x20'))**2).sqrt(),
            _5_4  = ((pl.col('x1')-pl.col('x21'))**2+(pl.col('x2')-pl.col('x22'))**2+(pl.col('x3')-pl.col('x23'))**2+(pl.col('x4')-pl.col('x24'))**2+(pl.col('x5')-pl.col('x25'))**2).sqrt(),
            _5_5  = ((pl.col('x1')-pl.col('x26'))**2+(pl.col('x2')-pl.col('x27'))**2+(pl.col('x3')-pl.col('x28'))**2+(pl.col('x4')-pl.col('x29'))**2+(pl.col('x5')-pl.col('x30'))**2).sqrt(),
            _5_6  = ((pl.col('x1')-pl.col('x31'))**2+(pl.col('x2')-pl.col('x32'))**2+(pl.col('x3')-pl.col('x33'))**2+(pl.col('x4')-pl.col('x34'))**2+(pl.col('x5')-pl.col('x35'))**2).sqrt(),
            _5_7  = ((pl.col('x1')-pl.col('x36'))**2+(pl.col('x2')-pl.col('x37'))**2+(pl.col('x3')-pl.col('x38'))**2+(pl.col('x4')-pl.col('x39'))**2+(pl.col('x5')-pl.col('x40'))**2).sqrt(),
            _5_8  = ((pl.col('x6')-pl.col('x11'))**2+(pl.col('x7')-pl.col('x12'))**2+(pl.col('x8')-pl.col('x13'))**2+(pl.col('x9')-pl.col('x14'))**2+(pl.col('x10')-pl.col('x15'))**2).sqrt(),
            _5_9  = ((pl.col('x6')-pl.col('x16'))**2+(pl.col('x7')-pl.col('x17'))**2+(pl.col('x8')-pl.col('x18'))**2+(pl.col('x9')-pl.col('x19'))**2+(pl.col('x10')-pl.col('x20'))**2).sqrt(),
            _5_10 = ((pl.col('x6')-pl.col('x21'))**2+(pl.col('x7')-pl.col('x22'))**2+(pl.col('x8')-pl.col('x23'))**2+(pl.col('x9')-pl.col('x24'))**2+(pl.col('x10')-pl.col('x25'))**2).sqrt(),
            _5_11 = ((pl.col('x6')-pl.col('x26'))**2+(pl.col('x7')-pl.col('x27'))**2+(pl.col('x8')-pl.col('x28'))**2+(pl.col('x9')-pl.col('x29'))**2+(pl.col('x10')-pl.col('x30'))**2).sqrt(),
            _5_12 = ((pl.col('x6')-pl.col('x31'))**2+(pl.col('x7')-pl.col('x32'))**2+(pl.col('x8')-pl.col('x33'))**2+(pl.col('x9')-pl.col('x34'))**2+(pl.col('x10')-pl.col('x35'))**2).sqrt(),
            _5_13 = ((pl.col('x6')-pl.col('x36'))**2+(pl.col('x7')-pl.col('x37'))**2+(pl.col('x8')-pl.col('x38'))**2+(pl.col('x9')-pl.col('x39'))**2+(pl.col('x10')-pl.col('x40'))**2).sqrt(),
            _5_14 = ((pl.col('x11')-pl.col('x16'))**2+(pl.col('x12')-pl.col('x17'))**2+(pl.col('x13')-pl.col('x18'))**2+(pl.col('x14')-pl.col('x19'))**2+(pl.col('x15')-pl.col('x20'))**2).sqrt(),
            _5_15 = ((pl.col('x11')-pl.col('x21'))**2+(pl.col('x12')-pl.col('x22'))**2+(pl.col('x13')-pl.col('x23'))**2+(pl.col('x14')-pl.col('x24'))**2+(pl.col('x15')-pl.col('x25'))**2).sqrt(),
            _5_16 = ((pl.col('x11')-pl.col('x26'))**2+(pl.col('x12')-pl.col('x27'))**2+(pl.col('x13')-pl.col('x28'))**2+(pl.col('x14')-pl.col('x29'))**2+(pl.col('x15')-pl.col('x30'))**2).sqrt(),
            _5_17 = ((pl.col('x11')-pl.col('x31'))**2+(pl.col('x12')-pl.col('x32'))**2+(pl.col('x13')-pl.col('x33'))**2+(pl.col('x14')-pl.col('x34'))**2+(pl.col('x15')-pl.col('x35'))**2).sqrt(),
            _5_18 = ((pl.col('x11')-pl.col('x36'))**2+(pl.col('x12')-pl.col('x37'))**2+(pl.col('x13')-pl.col('x38'))**2+(pl.col('x14')-pl.col('x39'))**2+(pl.col('x15')-pl.col('x40'))**2).sqrt(),
            _5_19 = ((pl.col('x16')-pl.col('x21'))**2+(pl.col('x17')-pl.col('x22'))**2+(pl.col('x18')-pl.col('x23'))**2+(pl.col('x19')-pl.col('x24'))**2+(pl.col('x20')-pl.col('x25'))**2).sqrt(),
            _5_20 = ((pl.col('x16')-pl.col('x26'))**2+(pl.col('x17')-pl.col('x27'))**2+(pl.col('x18')-pl.col('x28'))**2+(pl.col('x19')-pl.col('x29'))**2+(pl.col('x20')-pl.col('x30'))**2).sqrt(),
            _5_21 = ((pl.col('x16')-pl.col('x31'))**2+(pl.col('x17')-pl.col('x32'))**2+(pl.col('x18')-pl.col('x33'))**2+(pl.col('x19')-pl.col('x34'))**2+(pl.col('x20')-pl.col('x35'))**2).sqrt(),
            _5_22 = ((pl.col('x16')-pl.col('x36'))**2+(pl.col('x17')-pl.col('x37'))**2+(pl.col('x18')-pl.col('x38'))**2+(pl.col('x19')-pl.col('x39'))**2+(pl.col('x20')-pl.col('x40'))**2).sqrt(),
            _5_23 = ((pl.col('x21')-pl.col('x26'))**2+(pl.col('x22')-pl.col('x27'))**2+(pl.col('x23')-pl.col('x28'))**2+(pl.col('x24')-pl.col('x29'))**2+(pl.col('x25')-pl.col('x30'))**2).sqrt(),
            _5_24 = ((pl.col('x21')-pl.col('x31'))**2+(pl.col('x22')-pl.col('x32'))**2+(pl.col('x23')-pl.col('x33'))**2+(pl.col('x24')-pl.col('x34'))**2+(pl.col('x25')-pl.col('x35'))**2).sqrt(),
            _5_25 = ((pl.col('x21')-pl.col('x36'))**2+(pl.col('x22')-pl.col('x37'))**2+(pl.col('x23')-pl.col('x38'))**2+(pl.col('x24')-pl.col('x39'))**2+(pl.col('x25')-pl.col('x40'))**2).sqrt(),
            _5_26 = ((pl.col('x26')-pl.col('x31'))**2+(pl.col('x27')-pl.col('x32'))**2+(pl.col('x28')-pl.col('x33'))**2+(pl.col('x29')-pl.col('x34'))**2+(pl.col('x30')-pl.col('x35'))**2).sqrt(),
            _5_27 = ((pl.col('x26')-pl.col('x36'))**2+(pl.col('x27')-pl.col('x37'))**2+(pl.col('x28')-pl.col('x38'))**2+(pl.col('x29')-pl.col('x39'))**2+(pl.col('x30')-pl.col('x40'))**2).sqrt(),
            _5_28 = ((pl.col('x31')-pl.col('x36'))**2+(pl.col('x32')-pl.col('x37'))**2+(pl.col('x33')-pl.col('x38'))**2+(pl.col('x34')-pl.col('x39'))**2+(pl.col('x35')-pl.col('x40'))**2).sqrt(),
        )
        .with_columns(        #  = ((pl.col('    ')-pl.col('   '))**2+(pl.col('   '))**2).sqrt(),
            _4_1 = ((pl.col('x1')-pl.col('x5'))**2+(pl.col('x2')-pl.col('x6'))**2+(pl.col('x3')-pl.col('x7'))**2+(pl.col('x4')-pl.col('x8'))**2).sqrt(),    
            _4_2 = ((pl.col('x1')-pl.col('x9'))**2+(pl.col('x2')-pl.col('x10'))**2+(pl.col('x3')-pl.col('x11'))**2+(pl.col('x4')-pl.col('x12'))**2).sqrt(),
            _4_3 = ((pl.col('x1')-pl.col('x13'))**2+(pl.col('x2')-pl.col('x14'))**2+(pl.col('x3')-pl.col('x15'))**2+(pl.col('x4')-pl.col('x16'))**2).sqrt(),
            _4_4 = ((pl.col('x1')-pl.col('x17'))**2+(pl.col('x2')-pl.col('x18'))**2+(pl.col('x3')-pl.col('x19'))**2+(pl.col('x4')-pl.col('x20'))**2).sqrt(),
            _4_5 = ((pl.col('x1')-pl.col('x21'))**2+(pl.col('x2')-pl.col('x22'))**2+(pl.col('x3')-pl.col('x23'))**2+(pl.col('x4')-pl.col('x24'))**2).sqrt(),
            _4_6 = ((pl.col('x1')-pl.col('x25'))**2+(pl.col('x2')-pl.col('x26'))**2+(pl.col('x3')-pl.col('x27'))**2+(pl.col('x4')-pl.col('x28'))**2).sqrt(),
            _4_7 = ((pl.col('x1')-pl.col('x29'))**2+(pl.col('x2')-pl.col('x30'))**2+(pl.col('x3')-pl.col('x31'))**2+(pl.col('x4')-pl.col('x32'))**2).sqrt(),
            _4_8 = ((pl.col('x1')-pl.col('x33'))**2+(pl.col('x2')-pl.col('x34'))**2+(pl.col('x3')-pl.col('x35'))**2+(pl.col('x4')-pl.col('x36'))**2).sqrt(),
            _4_9 = ((pl.col('x1')-pl.col('x37'))**2+(pl.col('x2')-pl.col('x38'))**2+(pl.col('x3')-pl.col('x39'))**2+(pl.col('x4')-pl.col('x40'))**2).sqrt(),
            _4_10 = ((pl.col('x5')-pl.col('x9'))**2+(pl.col('x6')-pl.col('x10'))**2+(pl.col('x7')-pl.col('x11'))**2+(pl.col('x8')-pl.col('x12'))**2).sqrt(),
            _4_11 = ((pl.col('x5')-pl.col('x13'))**2+(pl.col('x6')-pl.col('x14'))**2+(pl.col('x7')-pl.col('x15'))**2+(pl.col('x8')-pl.col('x16'))**2).sqrt(),
            _4_12 = ((pl.col('x5')-pl.col('x17'))**2+(pl.col('x6')-pl.col('x18'))**2+(pl.col('x7')-pl.col('x19'))**2+(pl.col('x8')-pl.col('x20'))**2).sqrt(),
            _4_13 = ((pl.col('x5')-pl.col('x21'))**2+(pl.col('x6')-pl.col('x22'))**2+(pl.col('x7')-pl.col('x23'))**2+(pl.col('x8')-pl.col('x24'))**2).sqrt(),
            _4_14 = ((pl.col('x5')-pl.col('x25'))**2+(pl.col('x6')-pl.col('x26'))**2+(pl.col('x7')-pl.col('x27'))**2+(pl.col('x8')-pl.col('x28'))**2).sqrt(),
            _4_15 = ((pl.col('x5')-pl.col('x29'))**2+(pl.col('x6')-pl.col('x30'))**2+(pl.col('x7')-pl.col('x31'))**2+(pl.col('x8')-pl.col('x32'))**2).sqrt(),
            _4_16 = ((pl.col('x5')-pl.col('x33'))**2+(pl.col('x6')-pl.col('x34'))**2+(pl.col('x7')-pl.col('x35'))**2+(pl.col('x8')-pl.col('x36'))**2).sqrt(),
            _4_17 = ((pl.col('x5')-pl.col('x37'))**2+(pl.col('x6')-pl.col('x38'))**2+(pl.col('x7')-pl.col('x39'))**2+(pl.col('x8')-pl.col('x40'))**2).sqrt(),
            _4_18 = ((pl.col('x9')-pl.col('x13'))**2+(pl.col('x10')-pl.col('x14'))**2+(pl.col('x11')-pl.col('x15'))**2+(pl.col('x12')-pl.col('x16'))**2).sqrt(),
            _4_19 = ((pl.col('x9')-pl.col('x17'))**2+(pl.col('x10')-pl.col('x18'))**2+(pl.col('x11')-pl.col('x19'))**2+(pl.col('x12')-pl.col('x20'))**2).sqrt(),
            _4_20 = ((pl.col('x9')-pl.col('x21'))**2+(pl.col('x10')-pl.col('x22'))**2+(pl.col('x11')-pl.col('x23'))**2+(pl.col('x12')-pl.col('x24'))**2).sqrt(),
            _4_21 = ((pl.col('x9')-pl.col('x25'))**2+(pl.col('x10')-pl.col('x26'))**2+(pl.col('x11')-pl.col('x27'))**2+(pl.col('x12')-pl.col('x28'))**2).sqrt(),
            _4_22 = ((pl.col('x9')-pl.col('x29'))**2+(pl.col('x10')-pl.col('x30'))**2+(pl.col('x11')-pl.col('x31'))**2+(pl.col('x12')-pl.col('x32'))**2).sqrt(),
            _4_23 = ((pl.col('x9')-pl.col('x33'))**2+(pl.col('x10')-pl.col('x34'))**2+(pl.col('x11')-pl.col('x35'))**2+(pl.col('x12')-pl.col('x36'))**2).sqrt(),
            _4_24 = ((pl.col('x9')-pl.col('x37'))**2+(pl.col('x10')-pl.col('x38'))**2+(pl.col('x11')-pl.col('x39'))**2+(pl.col('x12')-pl.col('x40'))**2).sqrt(),
            _4_25 = ((pl.col('x13')-pl.col('x17'))**2+(pl.col('x14')-pl.col('x18'))**2+(pl.col('x15')-pl.col('x19'))**2+(pl.col('x16')-pl.col('x20'))**2).sqrt(),
            _4_26 = ((pl.col('x13')-pl.col('x21'))**2+(pl.col('x14')-pl.col('x22'))**2+(pl.col('x15')-pl.col('x23'))**2+(pl.col('x16')-pl.col('x24'))**2).sqrt(),
            _4_27 = ((pl.col('x13')-pl.col('x25'))**2+(pl.col('x14')-pl.col('x26'))**2+(pl.col('x15')-pl.col('x27'))**2+(pl.col('x16')-pl.col('x28'))**2).sqrt(),
            _4_28 = ((pl.col('x13')-pl.col('x29'))**2+(pl.col('x14')-pl.col('x30'))**2+(pl.col('x15')-pl.col('x31'))**2+(pl.col('x16')-pl.col('x32'))**2).sqrt(),
            _4_29 = ((pl.col('x13')-pl.col('x33'))**2+(pl.col('x14')-pl.col('x34'))**2+(pl.col('x15')-pl.col('x35'))**2+(pl.col('x16')-pl.col('x36'))**2).sqrt(),
            _4_30 = ((pl.col('x13')-pl.col('x37'))**2+(pl.col('x14')-pl.col('x38'))**2+(pl.col('x15')-pl.col('x39'))**2+(pl.col('x16')-pl.col('x40'))**2).sqrt(),
            _4_31 = ((pl.col('x17')-pl.col('x21'))**2+(pl.col('x18')-pl.col('x22'))**2+(pl.col('x19')-pl.col('x23'))**2+(pl.col('x20')-pl.col('x24'))**2).sqrt(),
            _4_32 = ((pl.col('x17')-pl.col('x25'))**2+(pl.col('x18')-pl.col('x26'))**2+(pl.col('x19')-pl.col('x27'))**2+(pl.col('x20')-pl.col('x28'))**2).sqrt(),
            _4_33 = ((pl.col('x17')-pl.col('x29'))**2+(pl.col('x18')-pl.col('x30'))**2+(pl.col('x19')-pl.col('x31'))**2+(pl.col('x20')-pl.col('x32'))**2).sqrt(),
            _4_34 = ((pl.col('x17')-pl.col('x33'))**2+(pl.col('x18')-pl.col('x34'))**2+(pl.col('x19')-pl.col('x35'))**2+(pl.col('x20')-pl.col('x36'))**2).sqrt(),
            _4_35 = ((pl.col('x17')-pl.col('x37'))**2+(pl.col('x18')-pl.col('x38'))**2+(pl.col('x19')-pl.col('x39'))**2+(pl.col('x20')-pl.col('x40'))**2).sqrt(),
            _4_36 = ((pl.col('x21')-pl.col('x25'))**2+(pl.col('x22')-pl.col('x26'))**2+(pl.col('x23')-pl.col('x27'))**2+(pl.col('x24')-pl.col('x28'))**2).sqrt(),
            _4_37 = ((pl.col('x21')-pl.col('x29'))**2+(pl.col('x22')-pl.col('x30'))**2+(pl.col('x23')-pl.col('x31'))**2+(pl.col('x24')-pl.col('x32'))**2).sqrt(),
            _4_38 = ((pl.col('x21')-pl.col('x33'))**2+(pl.col('x22')-pl.col('x34'))**2+(pl.col('x23')-pl.col('x35'))**2+(pl.col('x24')-pl.col('x36'))**2).sqrt(),
            _4_39 = ((pl.col('x21')-pl.col('x37'))**2+(pl.col('x22')-pl.col('x38'))**2+(pl.col('x23')-pl.col('x39'))**2+(pl.col('x24')-pl.col('x40'))**2).sqrt(),
            _4_40 = ((pl.col('x25')-pl.col('x29'))**2+(pl.col('x26')-pl.col('x30'))**2+(pl.col('x27')-pl.col('x31'))**2+(pl.col('x28')-pl.col('x32'))**2).sqrt(),
            _4_41 = ((pl.col('x25')-pl.col('x33'))**2+(pl.col('x26')-pl.col('x34'))**2+(pl.col('x27')-pl.col('x35'))**2+(pl.col('x28')-pl.col('x36'))**2).sqrt(),
            _4_42 = ((pl.col('x25')-pl.col('x37'))**2+(pl.col('x26')-pl.col('x38'))**2+(pl.col('x27')-pl.col('x39'))**2+(pl.col('x28')-pl.col('x40'))**2).sqrt(),
            _4_43 = ((pl.col('x29')-pl.col('x33'))**2+(pl.col('x30')-pl.col('x34'))**2+(pl.col('x31')-pl.col('x35'))**2+(pl.col('x32')-pl.col('x36'))**2).sqrt(),
            _4_44 = ((pl.col('x29')-pl.col('x37'))**2+(pl.col('x30')-pl.col('x38'))**2+(pl.col('x31')-pl.col('x39'))**2+(pl.col('x32')-pl.col('x40'))**2).sqrt(),
            _4_45 = ((pl.col('x33')-pl.col('x37'))**2+(pl.col('x34')-pl.col('x38'))**2+(pl.col('x35')-pl.col('x39'))**2+(pl.col('x36')-pl.col('x40'))**2).sqrt(),
        )
        .with_columns(
            ((pl.col(col) - pl.col(col).mean().over('patient_id')) / (pl.col(col).std().over('patient_id') + err)).alias(f'{col}_patient_norm') for col in (num_cols + new_num_cols)
        )
        .with_columns(
            count_per_patient = pl.col('isic_id').count().over('patient_id'),
        )
        .with_columns(
            pl.col(cat_cols).cast(pl.Categorical),
        )
        .to_pandas()
        .set_index(id_col)
    )

In [4]:
def preprocess(df_train, df_test):
    global cat_cols
    
    encoder = OneHotEncoder(sparse_output=False, dtype=np.int32, handle_unknown='ignore')
    encoder.fit(df_train[cat_cols])
    
    new_cat_cols = [f'onehot_{i}' for i in range(len(encoder.get_feature_names_out()))]

    df_train[new_cat_cols] = encoder.transform(df_train[cat_cols])
    df_train[new_cat_cols] = df_train[new_cat_cols].astype('category')

    df_test[new_cat_cols] = encoder.transform(df_test[cat_cols])
    df_test[new_cat_cols] = df_test[new_cat_cols].astype('category')

    for col in cat_cols:
        feature_cols.remove(col)

    feature_cols.extend(new_cat_cols)
    cat_cols = new_cat_cols
    
    return df_train, df_test

In [5]:
def custom_metric(estimator, X, y_true):
    y_hat = estimator.predict_proba(X)[:, 1]
    min_tpr = 0.80
    max_fpr = abs(1 - min_tpr)
    
    v_gt = abs(y_true - 1)
    v_pred = np.array([1.0 - x for x in y_hat])
    
    partial_auc_scaled = roc_auc_score(v_gt, v_pred, max_fpr=max_fpr)
    partial_auc = 0.5 * max_fpr**2 + (max_fpr - 0.5 * max_fpr**2) / (1.0 - 0.5) * (partial_auc_scaled - 0.5)
    
    return partial_auc

In [6]:
df_train = read_data(train_path)
df_test = read_data(test_path)
df_subm = pd.read_csv(subm_path, index_col=id_col)

df_train, df_test = preprocess(df_train, df_test)

In [ ]:
#不要な特徴量を削除
least_important_features = ['onehot_32', 'onehot_6', 'onehot_33', 'onehot_30', 'onehot_26', 'onehot_22', 'onehot_36', 'onehot_4']

df_train.drop(columns =least_important_features,inplace = True)
for feature in least_important_features:
    cat_cols.remove(feature)
    feature_cols.remove(feature)

In [8]:
lgb_params = {
    'objective':        'binary',
    'verbosity':        -1,
    'n_iter':           250,
    'boosting_type':    'gbdt',
    'random_state':     seed,
    'lambda_l1':        0.08758718919397321, 
    'lambda_l2':        0.0039689175176025465, 
    'learning_rate':    0.03231007103195577, 
    'max_depth':        4, 
    'num_leaves':       103, 
    'colsample_bytree': 0.8329551585827726, 
    'colsample_bynode': 0.4025961355653304, 
    'bagging_fraction': 0.7738954452473223, 
    'bagging_freq':     4, 
    'min_data_in_leaf': 85, 
    'scale_pos_weight': 2.7984184778875543,
}

lgb_model = Pipeline([
    ('sampler_1', RandomOverSampler(sampling_strategy= 0.003 , random_state=seed)),
    ('sampler_2', RandomUnderSampler(sampling_strategy=sampling_ratio, random_state=seed)),
    ('classifier', lgb.LGBMClassifier(**lgb_params)),
])

In [9]:
cb_params = {
    'loss_function':     'Logloss',
    'iterations':        250,
    'verbose':           False,
    'random_state':      seed,
    'max_depth':         7, 
    'learning_rate':     0.07, 
    'scale_pos_weight':  2.6149345838209532, 
    'l2_leaf_reg':       6.216113851699493, 
    'subsample':         0.6249261779711819, 
    'min_data_in_leaf':  24,
    'cat_features':      cat_cols,
}

cb_model = Pipeline([
    ('sampler_1', RandomOverSampler(sampling_strategy= 0.003 , random_state=seed)),
    ('sampler_2', RandomUnderSampler(sampling_strategy=sampling_ratio, random_state=seed)),
    ('classifier', cb.CatBoostClassifier(**cb_params)),
])

In [10]:
xgb_params = {
    'enable_categorical': True,
    'tree_method':        'hist',
    'random_state':       seed,
    'learning_rate':      0.08501257473292347, 
    'lambda':             8.879624125465703, 
    'alpha':              0.6779926606782505, 
    'max_depth':          6, 
    'subsample':          0.6012681388711075, 
    'colsample_bytree':   0.8437772277074493, 
    'colsample_bylevel':  0.5476090898823716, 
    'colsample_bynode':   0.9928601203635129, 
    'scale_pos_weight':   3.29440313334688,
}

xgb_model = Pipeline([
    ('sampler_1', RandomOverSampler(sampling_strategy= 0.003 , random_state=seed)),
    ('sampler_2', RandomUnderSampler(sampling_strategy=sampling_ratio, random_state=seed)),
    ('classifier', xgb.XGBClassifier(**xgb_params)),
])

In [11]:
estimator = VotingClassifier([
    ('lgb', lgb_model), ('cb', cb_model), ('xgb', xgb_model),
], voting='soft', weights=[0.45,0.35,0.20])

In [12]:
X = df_train[feature_cols]
y = df_train[target_col]
groups = df_train[group_col]
cv = StratifiedGroupKFold(5, shuffle=True, random_state=seed)

val_score = cross_val_score(
    estimator=estimator, 
    X=X, y=y, 
    cv=cv, 
    groups=groups,
    scoring=custom_metric,
)

np.mean(val_score), val_score

/opt/conda/lib/python3.10/site-packages/lightgbm/engine.py:172: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/opt/conda/lib/python3.10/site-packages/lightgbm/engine.py:172: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/opt/conda/lib/python3.10/site-packages/lightgbm/engine.py:172: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/opt/conda/lib/python3.10/site-packages/lightgbm/engine.py:172: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/opt/conda/lib/python3.10/site-packages/lightgbm/engine.py:172: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warnin

(0.17053557205191688,
 array([0.16749291, 0.16976978, 0.18426178, 0.16836154, 0.16279185]))

In [13]:
X, y = df_train[feature_cols], df_train[target_col]

estimator.fit(X, y)

/opt/conda/lib/python3.10/site-packages/lightgbm/engine.py:172: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


VotingClassifier(estimators=[('lgb',
                              Pipeline(steps=[('sampler_1',
                                               RandomOverSampler(random_state=42,
                                                                 sampling_strategy=0.003)),
                                              ('sampler_2',
                                               RandomUnderSampler(random_state=42,
                                                                  sampling_strategy=0.01)),
                                              ('classifier',
                                               LGBMClassifier(bagging_fraction=0.7738954452473223,
                                                              bagging_freq=4,
                                                              colsample_bynode=0.4025961355653304,
                                                              colsample_bytree=0.8329551585827726,...
                                                             interaction_constraints=None,
                                                             lambda=8.879624125465703,
                                                             learning_rate=0.08501257473292347,
                                                             max_bin=None,
                                                             max_cat_threshold=None,
                                                             max_cat_to_onehot=None,
                                                             max_delta_step=None,
                                                             max_depth=6,
                                                             max_leaves=None,
                                                             min_child_weight=None,
                                                             missing=nan,
                                                             monotone_constraints=None,
                                                             multi_strategy=None,
                                                             n_estimators=None,
                                                             n_jobs=None, ...))]))],
                 voting='soft', weights=[0.5, 0.4, 0.15])

In [ ]:
df_subm['target'] = estimator.predict_proba(df_test[feature_cols])[:, 1]

df_subm.to_csv('submission_1.csv')

In [15]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
import polars as pl

from sklearn.model_selection import StratifiedGroupKFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import VotingClassifier

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.pipeline import Pipeline

import lightgbm as lgb
import catboost as cb
import xgboost as xgb

import optuna

In [ ]:
root = Path('/kaggle/input/isic-2024-challenge')

train_path = root / 'train-metadata.csv'
test_path = root / 'test-metadata.csv'
subm_path = root / 'sample_submission.csv'

id_col = 'isic_id'
target_col = 'target'
group_col = 'patient_id'

err = 1e-5
sampling_ratio = 0.01
seed = 42

num_cols = [
    'age_approx',                        # Approximate age of patient at time of imaging.
    'clin_size_long_diam_mm',            # Maximum diameter of the lesion (mm).+
    'tbp_lv_A',                          # A inside  lesion.+
    'tbp_lv_Aext',                       # A outside lesion.+
    'tbp_lv_B',                          # B inside  lesion.+
    'tbp_lv_Bext',                       # B outside lesion.+ 
    'tbp_lv_C',                          # Chroma inside  lesion.+
    'tbp_lv_Cext',                       # Chroma outside lesion.+
    'tbp_lv_H',                          # Hue inside the lesion; calculated as the angle of A* and B* in LAB* color space. Typical values range from 25 (red) to 75 (brown).+
    'tbp_lv_Hext',                       # Hue outside lesion.+
    'tbp_lv_L',                          # L inside lesion.+
    'tbp_lv_Lext',                       # L outside lesion.+
    'tbp_lv_areaMM2',                    # Area of lesion (mm^2).+
    'tbp_lv_area_perim_ratio',           # Border jaggedness, the ratio between lesions perimeter and area. Circular lesions will have low values; irregular shaped lesions will have higher values. Values range 0-10.+
    'tbp_lv_color_std_mean',             # Color irregularity, calculated as the variance of colors within the lesion's boundary.
    'tbp_lv_deltaA',                     # Average A contrast (inside vs. outside lesion).+
    'tbp_lv_deltaB',                     # Average B contrast (inside vs. outside lesion).+
    'tbp_lv_deltaL',                     # Average L contrast (inside vs. outside lesion).+
    'tbp_lv_deltaLB',                    #
    'tbp_lv_deltaLBnorm',                # Contrast between the lesion and its immediate surrounding skin. Low contrast lesions tend to be faintly visible such as freckles; high contrast lesions tend to be those with darker pigment. Calculated as the average delta LB of the lesion relative to its immediate background in LAB* color space. Typical values range from 5.5 to 25.+
    'tbp_lv_eccentricity',               # Eccentricity.+
    'tbp_lv_minorAxisMM',                # Smallest lesion diameter (mm).+
    'tbp_lv_nevi_confidence',            # Nevus confidence score (0-100 scale) is a convolutional neural network classifier estimated probability that the lesion is a nevus. The neural network was trained on approximately 57,000 lesions that were classified and labeled by a dermatologist.+,++
    'tbp_lv_norm_border',                # Border irregularity (0-10 scale); the normalized average of border jaggedness and asymmetry.+
    'tbp_lv_norm_color',                 # Color variation (0-10 scale); the normalized average of color asymmetry and color irregularity.+
    'tbp_lv_perimeterMM',                # Perimeter of lesion (mm).+
    'tbp_lv_radial_color_std_max',       # Color asymmetry, a measure of asymmetry of the spatial distribution of color within the lesion. This score is calculated by looking at the average standard deviation in LAB* color space within concentric rings originating from the lesion center. Values range 0-10.+
    'tbp_lv_stdL',                       # Standard deviation of L inside  lesion.+
    'tbp_lv_stdLExt',                    # Standard deviation of L outside lesion.+
    'tbp_lv_symm_2axis',                 # Border asymmetry; a measure of asymmetry of the lesion's contour about an axis perpendicular to the lesion's most symmetric axis. Lesions with two axes of symmetry will therefore have low scores (more symmetric), while lesions with only one or zero axes of symmetry will have higher scores (less symmetric). This score is calculated by comparing opposite halves of the lesion contour over many degrees of rotation. The angle where the halves are most similar identifies the principal axis of symmetry, while the second axis of symmetry is perpendicular to the principal axis. Border asymmetry is reported as the asymmetry value about this second axis. Values range 0-10.+
    'tbp_lv_symm_2axis_angle',           # Lesion border asymmetry angle.+
    'tbp_lv_x',                          # X-coordinate of the lesion on 3D TBP.+
    'tbp_lv_y',                          # Y-coordinate of the lesion on 3D TBP.+
    'tbp_lv_z',                          # Z-coordinate of the lesion on 3D TBP.+
]

new_num_cols = [
    'x1',      # tbp_lv_minorAxisMM      / clin_size_long_diam_mm
    'x2',      # tbp_lv_areaMM2          / tbp_lv_perimeterMM **2
    'x3',      # tbp_lv_H                - tbp_lv_Hext              abs
    'x4',      # tbp_lv_L                - tbp_lv_Lext              abs
    'x5',      # tbp_lv_deltaA **2       + tbp_lv_deltaB **2 + tbp_lv_deltaL **2  sqrt  
    'x6',      # tbp_lv_norm_border      + tbp_lv_symm_2axis    
    'x7',      # tbp_lv_color_std_mean   / tbp_lv_radial_color_std_max
    'x8',      # tbp_lv_x **2 + tbp_lv_y **2 + tbp_lv_z **2  sqrt
    'x9',      # tbp_lv_perimeterMM      / tbp_lv_areaMM2
    'x10',     # tbp_lv_areaMM2          / tbp_lv_perimeterMM
    'x11',     # tbp_lv_deltaLBnorm      + tbp_lv_norm_color
    'x12',     # tbp_lv_symm_2axis       * tbp_lv_norm_border
    'x13',     # tbp_lv_symm_2axis       * tbp_lv_norm_border / (tbp_lv_symm_2axis + tbp_lv_norm_border)
    'x14',     # tbp_lv_stdL             / tbp_lv_Lext
    'x15',     # tbp_lv_stdL*tbp_lv_Lext / tbp_lv_stdL + tbp_lv_Lext
    'x16',     # clin_size_long_diam_mm  * age_approx
    'x17',     # tbp_lv_H                * tbp_lv_color_std_mean
    'x18',     # tbp_lv_norm_border      + tbp_lv_norm_color + tbp_lv_eccentricity / 3
    'x19',     # border_complexity       + lesion_shape_index
    'x20',     # tbp_lv_deltaA + tbp_lv_deltaB + tbp_lv_deltaL + tbp_lv_deltaLBnorm
    'x21',     # tbp_lv_areaMM2          + 1  np.log
    'x22',     # clin_size_long_diam_mm  / age_approx
    'x23',     # tbp_lv_H                + tbp_lv_Hext    / 2
    'x24',     # tbp_lv_deltaA **2 + tbp_lv_deltaB **2 + tbp_lv_deltaL **2   / 3  np.sqrt
    'x25',     # tbp_lv_color_std_mean   + bp_lv_area_perim_ratio + tbp_lv_symm_2axis   / 3
    'x26',     # tbp_lv_y                , tbp_lv_x  np.arctan2
    'x27',     # tbp_lv_deltaA           + tbp_lv_deltaB + tbp_lv_deltaL   / 3
    'x28',     # tbp_lv_symm_2axis       * tbp_lv_perimeterMM
    'x29',     # tbp_lv_area_perim_ratio + tbp_lv_eccentricity + bp_lv_norm_color + tbp_lv_symm_2axis   / 4
    'x30',     # tbp_lv_color_std_mean   / tbp_lv_stdLExt
    'x31',     # tbp_lv_norm_border      * tbp_lv_norm_color
    'x32',
    'x33',     # clin_size_long_diam_mm  / tbp_lv_deltaLBnorm
    'x34',     # tbp_lv_nevi_confidence  / age_approx
    'x35',
    'x36',     # tbp_lv_symm_2axis       * tbp_lv_radial_color_std_max
    'x37',     # tbp_lv_areaMM2          * sqrt(tbp_lv_x**2 + tbp_lv_y**2 + tbp_lv_z**2)
    'x38',     # abs(tbp_lv_L - tbp_lv_Lext) + abs(tbp_lv_A - tbp_lv_Aext) + abs(tbp_lv_B - tbp_lv_Bext)
    'x39',     # tbp_lv_eccentricity     * tbp_lv_color_std_mean
    'x40',     # tbp_lv_perimeterMM      / pi * sqrt(tbp_lv_areaMM2 / pi)
    'x41',     # age_approx              * clin_size_long_diam_mm * tbp_lv_symm_2axis
    'x42',     # age_approx              * tbp_lv_areaMM2 * tbp_lv_symm_2axis
]

new_num_cols2 = [    
    '_7_1',    # x1-x8   x2-x9   x3-x10  x4-x11  x5-x12  x6-x13  x7-x14
    '_7_2',    # x1-x15  x2-x16  x3-x17  x4-x18  x5-x19  x6-x20  x7-x21
    '_7_3',    # x1-x22  x2-x23  x3-x24  x4-x25  x5-x26  x6-x27  x7-x28
    '_7_4',    # x1-x29  x2-x30  x3-x31  x4-x32  x5-x33  x6-x34  x7-x35
    '_7_5',    # x1-x36  x2-x37  x3-x38  x4-x39  x5-x40  x6-x41  x7-x42
    '_7_6',    # x8-x15  x9-x16  x10-x17  x11-x18  x12-x19  x13-x20  x14-x21
    '_7_7',    # x8-x22  x9-x23  x10-x24  x11-x25  x12-x26  x13-x27  x14-x28
    '_7_8',    # x8-x29  x9-x30  x10-x31  x11-x32  x12-x33  x13-x34  x14-x35
    '_7_9',    # x8-x36  x9-x37  x10-x38  x11-x39  x12-x40  x13-x41  x14-x42
    '_7_10',   # x15-x22  x16-x23  x17-x24  x18-x25  x19-x26  x20-x27  x21-x28
    '_7_11',   # x15-x29  x16-x30  x17-x31  x18-x32  x19-x33  x20-x34  x21-x35
    '_7_12',   # x15-x36  x16-x37  x17-x38  x18-x39  x19-x40  x20-x41  x21-x42
    '_7_13',   # x22-x29  x23-x30  x24-x31  x25-x32  x26-x33  x27-x34  x28-x35
    '_7_14',   # x22-x36  x23-x37  x24-x38  x25-x39  x26-x40  x27-x41  x28-x42
    '_7_15',   # x29-x36  x30-x37  x31-x38  x32-x39  x33-x40  x34-x41  x35-x42
    
    '_6_1',    # x1-x7   x2-x8   x3-x9   x4-x10  x5-x11  x6-x12
    '_6_2',    # x1-x13  x2-x14  x3-x15  x4-x16  x5-x17  x6-x18
    '_6_3',    # x1-x19  x2-x20  x3-x21  x4-x22  x5-x23  x6-x24
    '_6_4',    # x1-x25  x2-x26  x3-x27  x4-x28  x5-x29  x6-x30
    '_6_5',    # x1-x31  x2-x32  x3-x33  x4-x34  x5-x35  x6-x36
    '_6_6',    # x1-x37  x2-x38  x3-x39  x4-x40  x5-x41  x6-x42
    '_6_7',    # x7-x13  x8-x14  x9-x15  x10-x16  x11-x17  x12-x23
    '_6_8',    # x7-x19  x8-x20  x9-x21  x10-x22  x11-x23  x12-x24
    '_6_9',    # x7-x25  x8-x26  x9-x27  x10-x28  x11-x29  x12-x30
    '_6_10',   # x7-x31  x8-x32  x9-x33  x10-x34  x11-x35  x12-x36
    '_6_11',   # x7-x37  x8-x38  x9-x39  x10-x40  x11-x41  x12-x42
    '_6_12',   # x13-x19  x14-x20  x15-x21  x16-x22  x17-x23  x18-x24
    '_6_13',   # x13-x25  x14-x26  x15-x27  x16-x28  x17-x29  x18-x30
    '_6_14',   # x13-x31  x14-x32  x15-x33  x16-x34  x17-x35  x18-x36
    '_6_15',   # x13-x37  x14-x38  x15-x39  x16-x40  x17-x41  x18-x42
    '_6_16',   # x19-x25  x20-x26  x21-x27  x22-x28  x23-x29  x24-x30
    '_6_17',   # x19-x31  x20-x32  x21-x33  x22-x34  x23-x35  x24-x36
    '_6_18',   # x19-x37  x20-x38  x21-x39  x22-x40  x23-x41  x24-x42
    '_6_19',   # x25-x31  x26-x32  x27-x33  x28-x34  x29-x35  x30-x36
    '_6_20',   # x25-x37  x26-x38  x27-x39  x28-x40  x29-x41  x30-x42
    '_6_21',   # x31-x37  x32-x38  x33-x39  x34-x40  x35-x41  x36-x42
    
    '_5_1',    # x1-x6    x2-x7    x3-x8    x4-x9    x5-x10
    '_5_2',    # x1-x11   x2-x12   x3-x13   x4-x14   x5-x15
    '_5_3',    # x1-x16   x2-x17   x3-x18   x4-x19   x5-x20
    '_5_4',    # x1-x21   x2-x22   x3-x23   x4-x24   x5-x25
    '_5_5',    # x1-x26   x2-x27   x3-x28   x4-x29   x5-x30
    '_5_6',    # x1-x31   x2-x32   x3-x33   x4-x34   x5-x35
    '_5_7',    # x1-x36   x2-x37   x3-x38   x4-x39   x5-x40
    '_5_8',    # x6-x11   x7-x12   x8-x13   x9-x14   x10-x15
    '_5_9',    # x6-x16   x7-x17   x8-x18   x9-x19   x10-x20
    '_5_10',   # x6-x21   x7-x22   x8-x23   x9-x24   x10-x25
    '_5_11',   # x6-x26   x7-x27   x8-x28   x9-x29   x10-x30
    '_5_12',   # x6-x31   x7-x32   x8-x33   x9-x34   x10-x35
    '_5_13',   # x6-x36   x7-x37   x8-x38   x9-x39   x10-x40
    '_5_14',   # x11-x16   x12-x17   x13-x18   x14-x19   x15-x20
    '_5_15',   # x11-x21   x12-x22   x13-x23   x14-x24   x15-x25
    '_5_16',   # x11-x26   x12-x27   x13-x28   x14-x29   x15-x30
    '_5_17',   # x11-x31   x12-x32   x13-x33   x14-x34   x15-x35
    '_5_18',   # x11-x36   x12-x37   x13-x38   x14-x39   x15-x40
    '_5_19',   # x16-x21   x17-x22   x18-x23   x19-x24   x20-x25
    '_5_20',   # x16-x26   x17-x27   x18-x28   x19-x29   x20-x30
    '_5_21',   # x16-x31   x17-x32   x18-x33   x19-x34   x20-x35
    '_5_22',   # x16-x36   x17-x37   x18-x38   x19-x39   x20-x40
    '_5_23',   # x21-x26   x22-x27   x23-x28   x24-x29   x25-x30
    '_5_24',   # x21-x31   x22-x32   x23-x33   x24-x34   x25-x35
    '_5_25',   # x21-x36   x22-x37   x23-x38   x24-x39   x25-x40
    '_5_26',   # x26-x31   x27-x32   x28-x33   x29-x34   x30-x35
    '_5_27',   # x26-x36   x27-x37   x28-x38   x29-x39   x30-x40
    '_5_28',   # x31-x36   x32-x37   x33-x38   x34-x39   x35-x40
    
    '_4_1',    # x1-x5    x2-x6    x3-x7    x4-x8    
    '_4_2',    # x1-x9    x2-x10   x3-x11   x4-x12
    '_4_3',    # x1-x13   x2-x14   x3-x15   x4-x16
    '_4_4',    # x1-x17   x2-x18   x3-x19   x4-x20
    '_4_5',    # x1-x21   x2-x22   x3-x23   x4-x24
    '_4_6',    # x1-x25   x2-x26   x3-x27   x4-x28
    '_4_7',    # x1-x29   x2-x30   x3-x31   x4-x32
    '_4_8',    # x1-x33   x2-x34   x3-x35   x4-x36
    '_4_9',    # x1-x37   x2-x38   x3-x39   x4-x40
    '_4_10',   # x5-x9    x6-x10   x7-x11   x8-x12
    '_4_11',   # x5-x13   x6-x14   x7-x15   x8-x16
    '_4_12',   # x5-x17   x6-x18   x7-x19   x8-x20
    '_4_13',   # x5-x21   x6-x22   x7-x23   x8-x24
    '_4_14',   # x5-x25   x6-x26   x7-x27   x8-x28
    '_4_15',   # x5-x29   x6-x30   x7-x31   x8-x32
    '_4_16',   # x5-x33   x6-x34   x7-x35   x8-x36
    '_4_17',   # x5-x37   x6-x38   x7-x39   x8-x40
    '_4_18',   # x9-x13   x10-x14   x11-x15   x12-x16
    '_4_19',   # x9-x17   x10-x18   x11-x19   x12-x20
    '_4_20',   # x9-x21   x10-x22   x11-x23   x12-x24
    '_4_21',   # x9-x25   x10-x26   x11-x27   x12-x28
    '_4_22',   # x9-x29   x10-x30   x11-x31   x12-x32
    '_4_23',   # x9-x33   x10-x34   x11-x35   x12-x36
    '_4_24',   # x9-x37   x10-x38   x11-x39   x12-x40
    '_4_25',   # x13-x17   x14-x18   x15-x19   x16-x20
    '_4_26',   # x13-x21   x14-x22   x15-x23   x16-x24
    '_4_27',   # x13-x25   x14-x26   x15-x27   x16-x28
    '_4_28',   # x13-x29   x14-x30   x15-x31   x16-x32
    '_4_29',   # x13-x33   x14-x34   x15-x35   x16-x36
    '_4_30',   # x13-x37   x14-x38   x15-x39   x16-x40
    '_4_31',   # x17-x21   x18-x22   x19-x23   x20-x24
    '_4_32',   # x17-x25   x18-x26   x19-x27   x20-x28
    '_4_33',   # x17-x29   x18-x30   x19-x31   x20-x32
    '_4_34',   # x17-x33   x18-x34   x19-x35   x20-x36
    '_4_35',   # x17-x37   x18-x38   x19-x39   x20-x40
    '_4_36',   # x21-x25   x22-x26   x23-x27   x24-x28
    '_4_37',   # x21-x29   x22-x30   x23-x31   x24-x32
    '_4_38',   # x21-x33   x22-x34   x23-x35   x24-x36
    '_4_39',   # x21-x37   x22-x38   x23-x39   x24-x40
    '_4_40',   # x25-x29   x26-x30   x27-x31   x28-x32
    '_4_41',   # x25-x33   x26-x34   x27-x35   x28-x36
    '_4_42',   # x25-x37   x26-x38   x27-x39   x28-x40
    '_4_43',   # x29-x33   x30-x34   x31-x35   x32-x36
    '_4_44',   # x29-x37   x30-x38   x31-x39   x32-x40
    '_4_45',   # x33-x37   x34-x38   x35-x39   x36-x40
]

cat_cols = ['sex', 'anatom_site_general', 'tbp_tile_type', 'tbp_lv_location', 'tbp_lv_location_simple', 'attribution']
norm_cols = [f'{col}_patient_norm' for col in num_cols + new_num_cols + new_num_cols2]
special_cols = ['count_per_patient']
feature_cols = num_cols + new_num_cols + new_num_cols2 + cat_cols + norm_cols + special_cols # 

In [ ]:
def read_data(path):
    return (
        pl.read_csv(path)
        .with_columns(
            pl.col('age_approx').cast(pl.String).replace('NA', np.nan).cast(pl.Float64),
        )
        .with_columns(
            pl.col(pl.Float64).fill_nan(pl.col(pl.Float64).median()), 
        )
        .with_columns(
            x1    = pl.col('tbp_lv_minorAxisMM') / pl.col('clin_size_long_diam_mm'),
            x2    = pl.col('tbp_lv_areaMM2') / (pl.col('tbp_lv_perimeterMM') ** 2),
            x3    = (pl.col('tbp_lv_H') - pl.col('tbp_lv_Hext')).abs(),
            x4    = (pl.col('tbp_lv_L') - pl.col('tbp_lv_Lext')).abs(),
            x5    = (pl.col('tbp_lv_deltaA') ** 2 + pl.col('tbp_lv_deltaB') ** 2 + pl.col('tbp_lv_deltaL') ** 2).sqrt(),
            x6    = pl.col('tbp_lv_norm_border') + pl.col('tbp_lv_symm_2axis'),
            x7    = pl.col('tbp_lv_color_std_mean') / (pl.col('tbp_lv_radial_color_std_max') + err),
        )
        .with_columns(
            x8    = (pl.col('tbp_lv_x') ** 2 + pl.col('tbp_lv_y') ** 2 + pl.col('tbp_lv_z') ** 2).sqrt(),
            x9    = pl.col('tbp_lv_perimeterMM') / pl.col('tbp_lv_areaMM2'),
            x10   = pl.col('tbp_lv_areaMM2') / pl.col('tbp_lv_perimeterMM'),
            x11   = pl.col('tbp_lv_deltaLBnorm') + pl.col('tbp_lv_norm_color'),
            xc1   = pl.col('anatom_site_general') + '_' + pl.col('tbp_lv_location'),
            x12   = pl.col('tbp_lv_symm_2axis') * pl.col('tbp_lv_norm_border'),
            x13   = pl.col('tbp_lv_symm_2axis') * pl.col('tbp_lv_norm_border') / (pl.col('tbp_lv_symm_2axis') + pl.col('tbp_lv_norm_border')),
        )
        .with_columns(
            x14   = pl.col('tbp_lv_stdL') / pl.col('tbp_lv_Lext'),
            x15   = pl.col('tbp_lv_stdL') * pl.col('tbp_lv_Lext') / (pl.col('tbp_lv_stdL') + pl.col('tbp_lv_Lext')),
            x16   = pl.col('clin_size_long_diam_mm') * pl.col('age_approx'),
            x17   = pl.col('tbp_lv_H') * pl.col('tbp_lv_color_std_mean'),
            x18   = (pl.col('tbp_lv_norm_border') + pl.col('tbp_lv_norm_color') + pl.col('tbp_lv_eccentricity')) / 3,
            x19   = pl.col('x6') + pl.col('x2'),
            x20   = pl.col('tbp_lv_deltaA') + pl.col('tbp_lv_deltaB') + pl.col('tbp_lv_deltaL') + pl.col('tbp_lv_deltaLBnorm'),
        )
        .with_columns(
            x21   = (pl.col('tbp_lv_areaMM2') + 1).log(),
            x22   = pl.col('clin_size_long_diam_mm') / pl.col('age_approx'),
            x23   = (pl.col('tbp_lv_H') + pl.col('tbp_lv_Hext')) / 2,
            x24   = ((pl.col('tbp_lv_deltaA') ** 2 + pl.col('tbp_lv_deltaB') ** 2 + pl.col('tbp_lv_deltaL') ** 2) / 3).sqrt(),
            x25   = (pl.col('tbp_lv_color_std_mean') + pl.col('tbp_lv_area_perim_ratio') + pl.col('tbp_lv_symm_2axis')) / 3,
            x26   = pl.arctan2(pl.col('tbp_lv_y'), pl.col('tbp_lv_x')),
            x27   = (pl.col('tbp_lv_deltaA') + pl.col('tbp_lv_deltaB') + pl.col('tbp_lv_deltaL')) / 3,
        )
        .with_columns(
            x28   = pl.col('tbp_lv_symm_2axis') * pl.col('tbp_lv_perimeterMM'),
            x29   = (pl.col('tbp_lv_area_perim_ratio') + pl.col('tbp_lv_eccentricity') + pl.col('tbp_lv_norm_color') + pl.col('tbp_lv_symm_2axis')) / 4,
            x30   = pl.col('tbp_lv_color_std_mean') / pl.col('tbp_lv_stdLExt'),
            x31   = pl.col('tbp_lv_norm_border') * pl.col('tbp_lv_norm_color'),
            x32   = pl.col('tbp_lv_norm_border') * pl.col('tbp_lv_norm_color') / (pl.col('tbp_lv_norm_border') + pl.col('tbp_lv_norm_color')),
            x33   = pl.col('clin_size_long_diam_mm') / pl.col('tbp_lv_deltaLBnorm'),
            x34   = pl.col('tbp_lv_nevi_confidence') / pl.col('age_approx'),
            x35   = (pl.col('clin_size_long_diam_mm')**2 + pl.col('age_approx')**2).sqrt(),
            x36   = pl.col('tbp_lv_radial_color_std_max') * pl.col('tbp_lv_symm_2axis'),
        )
        .with_columns(
            x37   = pl.col('tbp_lv_areaMM2') * (pl.col('tbp_lv_x')**2 + pl.col('tbp_lv_y')**2 + pl.col('tbp_lv_z')**2).sqrt(),
            x38   = (pl.col('tbp_lv_L') - pl.col('tbp_lv_Lext')).abs() + (pl.col('tbp_lv_A') - pl.col('tbp_lv_Aext')).abs() + (pl.col('tbp_lv_B') - pl.col('tbp_lv_Bext')).abs(),
            x39   = pl.col('tbp_lv_eccentricity') * pl.col('tbp_lv_color_std_mean'),
            x40   = pl.col('tbp_lv_perimeterMM') / (2 * np.pi * (pl.col('tbp_lv_areaMM2') / np.pi).sqrt()),
            x41   = pl.col('age_approx') * pl.col('clin_size_long_diam_mm') * pl.col('tbp_lv_symm_2axis'),
            x42   = pl.col('age_approx') * pl.col('tbp_lv_areaMM2') * pl.col('tbp_lv_symm_2axis'),
        )
        .with_columns(
            _7_1  = ((pl.col('x1')-pl.col('x8'))**2 +(pl.col('x2')-pl.col('x9'))**2 +(pl.col('x3')-pl.col('x10'))**2+(pl.col('x4')-pl.col('x11'))**2+(pl.col('x5')-pl.col('x12'))**2+(pl.col('x6')-pl.col('x13'))**2+(pl.col('x7')-pl.col('x14'))**2).sqrt(),
            _7_2  = ((pl.col('x1')-pl.col('x15'))**2+(pl.col('x2')-pl.col('x16'))**2+(pl.col('x3')-pl.col('x17'))**2+(pl.col('x4')-pl.col('x18'))**2+(pl.col('x5')-pl.col('x19'))**2+(pl.col('x6')-pl.col('x20'))**2+(pl.col('x7')-pl.col('x21'))**2).sqrt(),
            _7_3  = ((pl.col('x1')-pl.col('x22'))**2+(pl.col('x2')-pl.col('x23'))**2+(pl.col('x3')-pl.col('x24'))**2+(pl.col('x4')-pl.col('x25'))**2+(pl.col('x5')-pl.col('x26'))**2+(pl.col('x6')-pl.col('x27'))**2+(pl.col('x7')-pl.col('x28'))**2).sqrt(),
            _7_4  = ((pl.col('x1')-pl.col('x29'))**2+(pl.col('x2')-pl.col('x30'))**2+(pl.col('x3')-pl.col('x31'))**2+(pl.col('x4')-pl.col('x32'))**2+(pl.col('x5')-pl.col('x33'))**2+(pl.col('x6')-pl.col('x34'))**2+(pl.col('x7')-pl.col('x35'))**2).sqrt(),
            _7_5  = ((pl.col('x1')-pl.col('x36'))**2+(pl.col('x2')-pl.col('x37'))**2+(pl.col('x3')-pl.col('x38'))**2+(pl.col('x4')-pl.col('x39'))**2+(pl.col('x5')-pl.col('x40'))**2+(pl.col('x6')-pl.col('x41'))**2+(pl.col('x7')-pl.col('x42'))**2).sqrt(),
            _7_6  = ((pl.col('x8')-pl.col('x15'))**2+(pl.col('x9')-pl.col('x16'))**2+(pl.col('x10')-pl.col('x17'))**2+(pl.col('x11')-pl.col('x18'))**2+(pl.col('x12')-pl.col('x19'))**2+(pl.col('x13')-pl.col('x20'))**2+(pl.col('x14')-pl.col('x21'))**2).sqrt(),
            _7_7  = ((pl.col('x8')-pl.col('x22'))**2+(pl.col('x9')-pl.col('x23'))**2+(pl.col('x10')-pl.col('x24'))**2+(pl.col('x11')-pl.col('x25'))**2+(pl.col('x12')-pl.col('x26'))**2+(pl.col('x13')-pl.col('x27'))**2+(pl.col('x14')-pl.col('x28'))**2).sqrt(),
            _7_8  = ((pl.col('x8')-pl.col('x29'))**2+(pl.col('x9')-pl.col('x30'))**2+(pl.col('x10')-pl.col('x31'))**2+(pl.col('x11')-pl.col('x32'))**2+(pl.col('x12')-pl.col('x33'))**2+(pl.col('x13')-pl.col('x34'))**2+(pl.col('x14')-pl.col('x35'))**2).sqrt(),
            _7_9  = ((pl.col('x8')-pl.col('x36'))**2+(pl.col('x9')-pl.col('x37'))**2+(pl.col('x10')-pl.col('x38'))**2+(pl.col('x11')-pl.col('x39'))**2+(pl.col('x12')-pl.col('x40'))**2+(pl.col('x13')-pl.col('x41'))**2+(pl.col('x14')-pl.col('x42'))**2).sqrt(),
            _7_10 = ((pl.col('x15')-pl.col('x22'))**2+(pl.col('x16')-pl.col('x23'))**2+(pl.col('x17')-pl.col('x24'))**2+(pl.col('x18')-pl.col('x25'))**2+(pl.col('x19')-pl.col('x26'))**2+(pl.col('x20')-pl.col('x27'))**2+(pl.col('x21')-pl.col('x28'))**2).sqrt(),
            _7_11 = ((pl.col('x15')-pl.col('x29'))**2+(pl.col('x16')-pl.col('x30'))**2+(pl.col('x17')-pl.col('x31'))**2+(pl.col('x18')-pl.col('x32'))**2+(pl.col('x19')-pl.col('x33'))**2+(pl.col('x20')-pl.col('x34'))**2+(pl.col('x21')-pl.col('x35'))**2).sqrt(),
            _7_12 = ((pl.col('x15')-pl.col('x36'))**2+(pl.col('x16')-pl.col('x37'))**2+(pl.col('x17')-pl.col('x38'))**2+(pl.col('x18')-pl.col('x39'))**2+(pl.col('x19')-pl.col('x40'))**2+(pl.col('x20')-pl.col('x41'))**2+(pl.col('x21')-pl.col('x42'))**2).sqrt(),
            _7_13 = ((pl.col('x22')-pl.col('x29'))**2+(pl.col('x23')-pl.col('x30'))**2+(pl.col('x24')-pl.col('x31'))**2+(pl.col('x25')-pl.col('x32'))**2+(pl.col('x26')-pl.col('x33'))**2+(pl.col('x27')-pl.col('x34'))**2+(pl.col('x28')-pl.col('x35'))**2).sqrt(),
            _7_14 = ((pl.col('x22')-pl.col('x36'))**2+(pl.col('x23')-pl.col('x37'))**2+(pl.col('x24')-pl.col('x38'))**2+(pl.col('x25')-pl.col('x39'))**2+(pl.col('x26')-pl.col('x40'))**2+(pl.col('x27')-pl.col('x41'))**2+(pl.col('x28')-pl.col('x42'))**2).sqrt(),
            _7_15 = ((pl.col('x29')-pl.col('x36'))**2+(pl.col('x30')-pl.col('x37'))**2+(pl.col('x31')-pl.col('x38'))**2+(pl.col('x32')-pl.col('x39'))**2+(pl.col('x33')-pl.col('x40'))**2+(pl.col('x34')-pl.col('x41'))**2+(pl.col('x35')-pl.col('x42'))**2).sqrt(),
        )
                           
        .with_columns(
            _6_1  = ((pl.col('x1')-pl.col('x7'))**2+(pl.col('x2')-pl.col('x8'))**2+(pl.col('x3')-pl.col('x9'))**2+(pl.col('x4')-pl.col('x10'))**2+(pl.col('x5')-pl.col('x11'))**2+(pl.col('x6')-pl.col('x12'))**2).sqrt(),
            _6_2  = ((pl.col('x1')-pl.col('x13'))**2+(pl.col('x2')-pl.col('x14'))**2+(pl.col('x3')-pl.col('x15'))**2+(pl.col('x4')-pl.col('x16'))**2+(pl.col('x5')-pl.col('x17'))**2+(pl.col('x6')-pl.col('x18'))**2).sqrt(),
            _6_3  = ((pl.col('x1')-pl.col('x19'))**2+(pl.col('x2')-pl.col('x20'))**2+(pl.col('x3')-pl.col('x21'))**2+(pl.col('x4')-pl.col('x22'))**2+(pl.col('x5')-pl.col('x23'))**2+(pl.col('x6')-pl.col('x24'))**2).sqrt(),
            _6_4  = ((pl.col('x1')-pl.col('x25'))**2+(pl.col('x2')-pl.col('x26'))**2+(pl.col('x3')-pl.col('x27'))**2+(pl.col('x4')-pl.col('x28'))**2+(pl.col('x5')-pl.col('x29'))**2+(pl.col('x6')-pl.col('x30'))**2).sqrt(),
            _6_5  = ((pl.col('x1')-pl.col('x31'))**2+(pl.col('x2')-pl.col('x32'))**2+(pl.col('x3')-pl.col('x33'))**2+(pl.col('x4')-pl.col('x34'))**2+(pl.col('x5')-pl.col('x35'))**2+(pl.col('x6')-pl.col('x36'))**2).sqrt(),
            _6_6  = ((pl.col('x1')-pl.col('x37'))**2+(pl.col('x2')-pl.col('x38'))**2+(pl.col('x3')-pl.col('x39'))**2+(pl.col('x4')-pl.col('x40'))**2+(pl.col('x5')-pl.col('x41'))**2+(pl.col('x6')-pl.col('x42'))**2).sqrt(),
            _6_7  = ((pl.col('x7')-pl.col('x13'))**2+(pl.col('x8')-pl.col('x14'))**2+(pl.col('x9')-pl.col('x15'))**2+(pl.col('x10')-pl.col('x16'))**2+(pl.col('x11')-pl.col('x17'))**2+(pl.col('x12')-pl.col('x23'))**2).sqrt(),
            _6_8  = ((pl.col('x7')-pl.col('x19'))**2+(pl.col('x8')-pl.col('x20'))**2+(pl.col('x9')-pl.col('x21'))**2+(pl.col('x10')-pl.col('x22'))**2+(pl.col('x11')-pl.col('x23'))**2+(pl.col('x12')-pl.col('x24'))**2).sqrt(),
            _6_9  = ((pl.col('x7')-pl.col('x25'))**2+(pl.col('x8')-pl.col('x26'))**2+(pl.col('x9')-pl.col('x27'))**2+(pl.col('x10')-pl.col('x28'))**2+(pl.col('x11')-pl.col('x29'))**2+(pl.col('x12')-pl.col('x30'))**2).sqrt(),
            _6_10 = ((pl.col('x7')-pl.col('x31'))**2+(pl.col('x8')-pl.col('x32'))**2+(pl.col('x9')-pl.col('x33'))**2+(pl.col('x10')-pl.col('x34'))**2+(pl.col('x11')-pl.col('x35'))**2+(pl.col('x12')-pl.col('x36'))**2).sqrt(),
            _6_11 = ((pl.col('x7')-pl.col('x37'))**2+(pl.col('x8')-pl.col('x38'))**2+(pl.col('x9')-pl.col('x39'))**2+(pl.col('x10')-pl.col('x40'))**2+(pl.col('x11')-pl.col('x41'))**2+(pl.col('x12')-pl.col('x42'))**2).sqrt(),
            _6_12 = ((pl.col('x13')-pl.col('x19'))**2+(pl.col('x14')-pl.col('x20'))**2+(pl.col('x15')-pl.col('x21'))**2+(pl.col('x16')-pl.col('x22'))**2+(pl.col('x17')-pl.col('x23'))**2+(pl.col('x18')-pl.col('x24'))**2).sqrt(),
            _6_13 = ((pl.col('x13')-pl.col('x25'))**2+(pl.col('x14')-pl.col('x26'))**2+(pl.col('x15')-pl.col('x27'))**2+(pl.col('x16')-pl.col('x28'))**2+(pl.col('x17')-pl.col('x29'))**2+(pl.col('x18')-pl.col('x30'))**2).sqrt(),
            _6_14 = ((pl.col('x13')-pl.col('x31'))**2+(pl.col('x14')-pl.col('x32'))**2+(pl.col('x15')-pl.col('x33'))**2+(pl.col('x16')-pl.col('x34'))**2+(pl.col('x17')-pl.col('x35'))**2+(pl.col('x18')-pl.col('x36'))**2).sqrt(),
            _6_15 = ((pl.col('x13')-pl.col('x37'))**2+(pl.col('x14')-pl.col('x38'))**2+(pl.col('x15')-pl.col('x39'))**2+(pl.col('x16')-pl.col('x40'))**2+(pl.col('x17')-pl.col('x41'))**2+(pl.col('x18')-pl.col('x42'))**2).sqrt(),
            _6_16 = ((pl.col('x19')-pl.col('x25'))**2+(pl.col('x20')-pl.col('x26'))**2+(pl.col('x21')-pl.col('x27'))**2+(pl.col('x22')-pl.col('x28'))**2+(pl.col('x23')-pl.col('x29'))**2+(pl.col('x24')-pl.col('x30'))**2).sqrt(),
            _6_17 = ((pl.col('x19')-pl.col('x31'))**2+(pl.col('x20')-pl.col('x32'))**2+(pl.col('x21')-pl.col('x33'))**2+(pl.col('x22')-pl.col('x34'))**2+(pl.col('x23')-pl.col('x35'))**2+(pl.col('x24')-pl.col('x36'))**2).sqrt(),
            _6_18 = ((pl.col('x19')-pl.col('x37'))**2+(pl.col('x20')-pl.col('x38'))**2+(pl.col('x21')-pl.col('x39'))**2+(pl.col('x22')-pl.col('x40'))**2+(pl.col('x23')-pl.col('x41'))**2+(pl.col('x24')-pl.col('x42'))**2).sqrt(),
            _6_19 = ((pl.col('x25')-pl.col('x31'))**2+(pl.col('x26')-pl.col('x32'))**2+(pl.col('x27')-pl.col('x33'))**2+(pl.col('x28')-pl.col('x34'))**2+(pl.col('x29')-pl.col('x35'))**2+(pl.col('x30')-pl.col('x36'))**2).sqrt(),
            _6_20 = ((pl.col('x25')-pl.col('x37'))**2+(pl.col('x26')-pl.col('x38'))**2+(pl.col('x27')-pl.col('x39'))**2+(pl.col('x28')-pl.col('x40'))**2+(pl.col('x29')-pl.col('x41'))**2+(pl.col('x30')-pl.col('x42'))**2).sqrt(),
            _6_21 = ((pl.col('x31')-pl.col('x37'))**2+(pl.col('x32')-pl.col('x38'))**2+(pl.col('x33')-pl.col('x39'))**2+(pl.col('x34')-pl.col('x40'))**2+(pl.col('x35')-pl.col('x41'))**2+(pl.col('x36')-pl.col('x42'))**2).sqrt(),
        )                    
# #                            
        .with_columns(  
            _5_1  = ((pl.col('x1')-pl.col('x6'))**2 +(pl.col('x2')-pl.col('x7'))**2 +(pl.col('x3')-pl.col('x8'))**2 +(pl.col('x4')-pl.col('x9'))**2 +(pl.col('x5')-pl.col('x10'))**2).sqrt(),
            _5_2  = ((pl.col('x1')-pl.col('x11'))**2+(pl.col('x2')-pl.col('x12'))**2+(pl.col('x3')-pl.col('x13'))**2+(pl.col('x4')-pl.col('x14'))**2+(pl.col('x5')-pl.col('x15'))**2).sqrt(),
            _5_3  = ((pl.col('x1')-pl.col('x16'))**2+(pl.col('x2')-pl.col('x17'))**2+(pl.col('x3')-pl.col('x18'))**2+(pl.col('x4')-pl.col('x19'))**2+(pl.col('x5')-pl.col('x20'))**2).sqrt(),
            _5_4  = ((pl.col('x1')-pl.col('x21'))**2+(pl.col('x2')-pl.col('x22'))**2+(pl.col('x3')-pl.col('x23'))**2+(pl.col('x4')-pl.col('x24'))**2+(pl.col('x5')-pl.col('x25'))**2).sqrt(),
            _5_5  = ((pl.col('x1')-pl.col('x26'))**2+(pl.col('x2')-pl.col('x27'))**2+(pl.col('x3')-pl.col('x28'))**2+(pl.col('x4')-pl.col('x29'))**2+(pl.col('x5')-pl.col('x30'))**2).sqrt(),
            _5_6  = ((pl.col('x1')-pl.col('x31'))**2+(pl.col('x2')-pl.col('x32'))**2+(pl.col('x3')-pl.col('x33'))**2+(pl.col('x4')-pl.col('x34'))**2+(pl.col('x5')-pl.col('x35'))**2).sqrt(),
            _5_7  = ((pl.col('x1')-pl.col('x36'))**2+(pl.col('x2')-pl.col('x37'))**2+(pl.col('x3')-pl.col('x38'))**2+(pl.col('x4')-pl.col('x39'))**2+(pl.col('x5')-pl.col('x40'))**2).sqrt(),
            _5_8  = ((pl.col('x6')-pl.col('x11'))**2+(pl.col('x7')-pl.col('x12'))**2+(pl.col('x8')-pl.col('x13'))**2+(pl.col('x9')-pl.col('x14'))**2+(pl.col('x10')-pl.col('x15'))**2).sqrt(),
            _5_9  = ((pl.col('x6')-pl.col('x16'))**2+(pl.col('x7')-pl.col('x17'))**2+(pl.col('x8')-pl.col('x18'))**2+(pl.col('x9')-pl.col('x19'))**2+(pl.col('x10')-pl.col('x20'))**2).sqrt(),
            _5_10 = ((pl.col('x6')-pl.col('x21'))**2+(pl.col('x7')-pl.col('x22'))**2+(pl.col('x8')-pl.col('x23'))**2+(pl.col('x9')-pl.col('x24'))**2+(pl.col('x10')-pl.col('x25'))**2).sqrt(),
            _5_11 = ((pl.col('x6')-pl.col('x26'))**2+(pl.col('x7')-pl.col('x27'))**2+(pl.col('x8')-pl.col('x28'))**2+(pl.col('x9')-pl.col('x29'))**2+(pl.col('x10')-pl.col('x30'))**2).sqrt(),
            _5_12 = ((pl.col('x6')-pl.col('x31'))**2+(pl.col('x7')-pl.col('x32'))**2+(pl.col('x8')-pl.col('x33'))**2+(pl.col('x9')-pl.col('x34'))**2+(pl.col('x10')-pl.col('x35'))**2).sqrt(),
            _5_13 = ((pl.col('x6')-pl.col('x36'))**2+(pl.col('x7')-pl.col('x37'))**2+(pl.col('x8')-pl.col('x38'))**2+(pl.col('x9')-pl.col('x39'))**2+(pl.col('x10')-pl.col('x40'))**2).sqrt(),
            _5_14 = ((pl.col('x11')-pl.col('x16'))**2+(pl.col('x12')-pl.col('x17'))**2+(pl.col('x13')-pl.col('x18'))**2+(pl.col('x14')-pl.col('x19'))**2+(pl.col('x15')-pl.col('x20'))**2).sqrt(),
            _5_15 = ((pl.col('x11')-pl.col('x21'))**2+(pl.col('x12')-pl.col('x22'))**2+(pl.col('x13')-pl.col('x23'))**2+(pl.col('x14')-pl.col('x24'))**2+(pl.col('x15')-pl.col('x25'))**2).sqrt(),
            _5_16 = ((pl.col('x11')-pl.col('x26'))**2+(pl.col('x12')-pl.col('x27'))**2+(pl.col('x13')-pl.col('x28'))**2+(pl.col('x14')-pl.col('x29'))**2+(pl.col('x15')-pl.col('x30'))**2).sqrt(),
            _5_17 = ((pl.col('x11')-pl.col('x31'))**2+(pl.col('x12')-pl.col('x32'))**2+(pl.col('x13')-pl.col('x33'))**2+(pl.col('x14')-pl.col('x34'))**2+(pl.col('x15')-pl.col('x35'))**2).sqrt(),
            _5_18 = ((pl.col('x11')-pl.col('x36'))**2+(pl.col('x12')-pl.col('x37'))**2+(pl.col('x13')-pl.col('x38'))**2+(pl.col('x14')-pl.col('x39'))**2+(pl.col('x15')-pl.col('x40'))**2).sqrt(),
            _5_19 = ((pl.col('x16')-pl.col('x21'))**2+(pl.col('x17')-pl.col('x22'))**2+(pl.col('x18')-pl.col('x23'))**2+(pl.col('x19')-pl.col('x24'))**2+(pl.col('x20')-pl.col('x25'))**2).sqrt(),
            _5_20 = ((pl.col('x16')-pl.col('x26'))**2+(pl.col('x17')-pl.col('x27'))**2+(pl.col('x18')-pl.col('x28'))**2+(pl.col('x19')-pl.col('x29'))**2+(pl.col('x20')-pl.col('x30'))**2).sqrt(),
            _5_21 = ((pl.col('x16')-pl.col('x31'))**2+(pl.col('x17')-pl.col('x32'))**2+(pl.col('x18')-pl.col('x33'))**2+(pl.col('x19')-pl.col('x34'))**2+(pl.col('x20')-pl.col('x35'))**2).sqrt(),
            _5_22 = ((pl.col('x16')-pl.col('x36'))**2+(pl.col('x17')-pl.col('x37'))**2+(pl.col('x18')-pl.col('x38'))**2+(pl.col('x19')-pl.col('x39'))**2+(pl.col('x20')-pl.col('x40'))**2).sqrt(),
            _5_23 = ((pl.col('x21')-pl.col('x26'))**2+(pl.col('x22')-pl.col('x27'))**2+(pl.col('x23')-pl.col('x28'))**2+(pl.col('x24')-pl.col('x29'))**2+(pl.col('x25')-pl.col('x30'))**2).sqrt(),
            _5_24 = ((pl.col('x21')-pl.col('x31'))**2+(pl.col('x22')-pl.col('x32'))**2+(pl.col('x23')-pl.col('x33'))**2+(pl.col('x24')-pl.col('x34'))**2+(pl.col('x25')-pl.col('x35'))**2).sqrt(),
            _5_25 = ((pl.col('x21')-pl.col('x36'))**2+(pl.col('x22')-pl.col('x37'))**2+(pl.col('x23')-pl.col('x38'))**2+(pl.col('x24')-pl.col('x39'))**2+(pl.col('x25')-pl.col('x40'))**2).sqrt(),
            _5_26 = ((pl.col('x26')-pl.col('x31'))**2+(pl.col('x27')-pl.col('x32'))**2+(pl.col('x28')-pl.col('x33'))**2+(pl.col('x29')-pl.col('x34'))**2+(pl.col('x30')-pl.col('x35'))**2).sqrt(),
            _5_27 = ((pl.col('x26')-pl.col('x36'))**2+(pl.col('x27')-pl.col('x37'))**2+(pl.col('x28')-pl.col('x38'))**2+(pl.col('x29')-pl.col('x39'))**2+(pl.col('x30')-pl.col('x40'))**2).sqrt(),
            _5_28 = ((pl.col('x31')-pl.col('x36'))**2+(pl.col('x32')-pl.col('x37'))**2+(pl.col('x33')-pl.col('x38'))**2+(pl.col('x34')-pl.col('x39'))**2+(pl.col('x35')-pl.col('x40'))**2).sqrt(),
        )
        .with_columns(       
            _4_1 = ((pl.col('x1')-pl.col('x5'))**2+(pl.col('x2')-pl.col('x6'))**2+(pl.col('x3')-pl.col('x7'))**2+(pl.col('x4')-pl.col('x8'))**2).sqrt(),    
            _4_2 = ((pl.col('x1')-pl.col('x9'))**2+(pl.col('x2')-pl.col('x10'))**2+(pl.col('x3')-pl.col('x11'))**2+(pl.col('x4')-pl.col('x12'))**2).sqrt(),
            _4_3 = ((pl.col('x1')-pl.col('x13'))**2+(pl.col('x2')-pl.col('x14'))**2+(pl.col('x3')-pl.col('x15'))**2+(pl.col('x4')-pl.col('x16'))**2).sqrt(),
            _4_4 = ((pl.col('x1')-pl.col('x17'))**2+(pl.col('x2')-pl.col('x18'))**2+(pl.col('x3')-pl.col('x19'))**2+(pl.col('x4')-pl.col('x20'))**2).sqrt(),
            _4_5 = ((pl.col('x1')-pl.col('x21'))**2+(pl.col('x2')-pl.col('x22'))**2+(pl.col('x3')-pl.col('x23'))**2+(pl.col('x4')-pl.col('x24'))**2).sqrt(),
            _4_6 = ((pl.col('x1')-pl.col('x25'))**2+(pl.col('x2')-pl.col('x26'))**2+(pl.col('x3')-pl.col('x27'))**2+(pl.col('x4')-pl.col('x28'))**2).sqrt(),
            _4_7 = ((pl.col('x1')-pl.col('x29'))**2+(pl.col('x2')-pl.col('x30'))**2+(pl.col('x3')-pl.col('x31'))**2+(pl.col('x4')-pl.col('x32'))**2).sqrt(),
            _4_8 = ((pl.col('x1')-pl.col('x33'))**2+(pl.col('x2')-pl.col('x34'))**2+(pl.col('x3')-pl.col('x35'))**2+(pl.col('x4')-pl.col('x36'))**2).sqrt(),
            _4_9 = ((pl.col('x1')-pl.col('x37'))**2+(pl.col('x2')-pl.col('x38'))**2+(pl.col('x3')-pl.col('x39'))**2+(pl.col('x4')-pl.col('x40'))**2).sqrt(),
            _4_10 = ((pl.col('x5')-pl.col('x9'))**2+(pl.col('x6')-pl.col('x10'))**2+(pl.col('x7')-pl.col('x11'))**2+(pl.col('x8')-pl.col('x12'))**2).sqrt(),
            _4_11 = ((pl.col('x5')-pl.col('x13'))**2+(pl.col('x6')-pl.col('x14'))**2+(pl.col('x7')-pl.col('x15'))**2+(pl.col('x8')-pl.col('x16'))**2).sqrt(),
            _4_12 = ((pl.col('x5')-pl.col('x17'))**2+(pl.col('x6')-pl.col('x18'))**2+(pl.col('x7')-pl.col('x19'))**2+(pl.col('x8')-pl.col('x20'))**2).sqrt(),
            _4_13 = ((pl.col('x5')-pl.col('x21'))**2+(pl.col('x6')-pl.col('x22'))**2+(pl.col('x7')-pl.col('x23'))**2+(pl.col('x8')-pl.col('x24'))**2).sqrt(),
            _4_14 = ((pl.col('x5')-pl.col('x25'))**2+(pl.col('x6')-pl.col('x26'))**2+(pl.col('x7')-pl.col('x27'))**2+(pl.col('x8')-pl.col('x28'))**2).sqrt(),
            _4_15 = ((pl.col('x5')-pl.col('x29'))**2+(pl.col('x6')-pl.col('x30'))**2+(pl.col('x7')-pl.col('x31'))**2+(pl.col('x8')-pl.col('x32'))**2).sqrt(),
            _4_16 = ((pl.col('x5')-pl.col('x33'))**2+(pl.col('x6')-pl.col('x34'))**2+(pl.col('x7')-pl.col('x35'))**2+(pl.col('x8')-pl.col('x36'))**2).sqrt(),
            _4_17 = ((pl.col('x5')-pl.col('x37'))**2+(pl.col('x6')-pl.col('x38'))**2+(pl.col('x7')-pl.col('x39'))**2+(pl.col('x8')-pl.col('x40'))**2).sqrt(),
            _4_18 = ((pl.col('x9')-pl.col('x13'))**2+(pl.col('x10')-pl.col('x14'))**2+(pl.col('x11')-pl.col('x15'))**2+(pl.col('x12')-pl.col('x16'))**2).sqrt(),
            _4_19 = ((pl.col('x9')-pl.col('x17'))**2+(pl.col('x10')-pl.col('x18'))**2+(pl.col('x11')-pl.col('x19'))**2+(pl.col('x12')-pl.col('x20'))**2).sqrt(),
            _4_20 = ((pl.col('x9')-pl.col('x21'))**2+(pl.col('x10')-pl.col('x22'))**2+(pl.col('x11')-pl.col('x23'))**2+(pl.col('x12')-pl.col('x24'))**2).sqrt(),
            _4_21 = ((pl.col('x9')-pl.col('x25'))**2+(pl.col('x10')-pl.col('x26'))**2+(pl.col('x11')-pl.col('x27'))**2+(pl.col('x12')-pl.col('x28'))**2).sqrt(),
            _4_22 = ((pl.col('x9')-pl.col('x29'))**2+(pl.col('x10')-pl.col('x30'))**2+(pl.col('x11')-pl.col('x31'))**2+(pl.col('x12')-pl.col('x32'))**2).sqrt(),
            _4_23 = ((pl.col('x9')-pl.col('x33'))**2+(pl.col('x10')-pl.col('x34'))**2+(pl.col('x11')-pl.col('x35'))**2+(pl.col('x12')-pl.col('x36'))**2).sqrt(),
            _4_24 = ((pl.col('x9')-pl.col('x37'))**2+(pl.col('x10')-pl.col('x38'))**2+(pl.col('x11')-pl.col('x39'))**2+(pl.col('x12')-pl.col('x40'))**2).sqrt(),
            _4_25 = ((pl.col('x13')-pl.col('x17'))**2+(pl.col('x14')-pl.col('x18'))**2+(pl.col('x15')-pl.col('x19'))**2+(pl.col('x16')-pl.col('x20'))**2).sqrt(),
            _4_26 = ((pl.col('x13')-pl.col('x21'))**2+(pl.col('x14')-pl.col('x22'))**2+(pl.col('x15')-pl.col('x23'))**2+(pl.col('x16')-pl.col('x24'))**2).sqrt(),
            _4_27 = ((pl.col('x13')-pl.col('x25'))**2+(pl.col('x14')-pl.col('x26'))**2+(pl.col('x15')-pl.col('x27'))**2+(pl.col('x16')-pl.col('x28'))**2).sqrt(),
            _4_28 = ((pl.col('x13')-pl.col('x29'))**2+(pl.col('x14')-pl.col('x30'))**2+(pl.col('x15')-pl.col('x31'))**2+(pl.col('x16')-pl.col('x32'))**2).sqrt(),
            _4_29 = ((pl.col('x13')-pl.col('x33'))**2+(pl.col('x14')-pl.col('x34'))**2+(pl.col('x15')-pl.col('x35'))**2+(pl.col('x16')-pl.col('x36'))**2).sqrt(),
            _4_30 = ((pl.col('x13')-pl.col('x37'))**2+(pl.col('x14')-pl.col('x38'))**2+(pl.col('x15')-pl.col('x39'))**2+(pl.col('x16')-pl.col('x40'))**2).sqrt(),
            _4_31 = ((pl.col('x17')-pl.col('x21'))**2+(pl.col('x18')-pl.col('x22'))**2+(pl.col('x19')-pl.col('x23'))**2+(pl.col('x20')-pl.col('x24'))**2).sqrt(),
            _4_32 = ((pl.col('x17')-pl.col('x25'))**2+(pl.col('x18')-pl.col('x26'))**2+(pl.col('x19')-pl.col('x27'))**2+(pl.col('x20')-pl.col('x28'))**2).sqrt(),
            _4_33 = ((pl.col('x17')-pl.col('x29'))**2+(pl.col('x18')-pl.col('x30'))**2+(pl.col('x19')-pl.col('x31'))**2+(pl.col('x20')-pl.col('x32'))**2).sqrt(),
            _4_34 = ((pl.col('x17')-pl.col('x33'))**2+(pl.col('x18')-pl.col('x34'))**2+(pl.col('x19')-pl.col('x35'))**2+(pl.col('x20')-pl.col('x36'))**2).sqrt(),
            _4_35 = ((pl.col('x17')-pl.col('x37'))**2+(pl.col('x18')-pl.col('x38'))**2+(pl.col('x19')-pl.col('x39'))**2+(pl.col('x20')-pl.col('x40'))**2).sqrt(),
            _4_36 = ((pl.col('x21')-pl.col('x25'))**2+(pl.col('x22')-pl.col('x26'))**2+(pl.col('x23')-pl.col('x27'))**2+(pl.col('x24')-pl.col('x28'))**2).sqrt(),
            _4_37 = ((pl.col('x21')-pl.col('x29'))**2+(pl.col('x22')-pl.col('x30'))**2+(pl.col('x23')-pl.col('x31'))**2+(pl.col('x24')-pl.col('x32'))**2).sqrt(),
            _4_38 = ((pl.col('x21')-pl.col('x33'))**2+(pl.col('x22')-pl.col('x34'))**2+(pl.col('x23')-pl.col('x35'))**2+(pl.col('x24')-pl.col('x36'))**2).sqrt(),
            _4_39 = ((pl.col('x21')-pl.col('x37'))**2+(pl.col('x22')-pl.col('x38'))**2+(pl.col('x23')-pl.col('x39'))**2+(pl.col('x24')-pl.col('x40'))**2).sqrt(),
            _4_40 = ((pl.col('x25')-pl.col('x29'))**2+(pl.col('x26')-pl.col('x30'))**2+(pl.col('x27')-pl.col('x31'))**2+(pl.col('x28')-pl.col('x32'))**2).sqrt(),
            _4_41 = ((pl.col('x25')-pl.col('x33'))**2+(pl.col('x26')-pl.col('x34'))**2+(pl.col('x27')-pl.col('x35'))**2+(pl.col('x28')-pl.col('x36'))**2).sqrt(),
            _4_42 = ((pl.col('x25')-pl.col('x37'))**2+(pl.col('x26')-pl.col('x38'))**2+(pl.col('x27')-pl.col('x39'))**2+(pl.col('x28')-pl.col('x40'))**2).sqrt(),
            _4_43 = ((pl.col('x29')-pl.col('x33'))**2+(pl.col('x30')-pl.col('x34'))**2+(pl.col('x31')-pl.col('x35'))**2+(pl.col('x32')-pl.col('x36'))**2).sqrt(),
            _4_44 = ((pl.col('x29')-pl.col('x37'))**2+(pl.col('x30')-pl.col('x38'))**2+(pl.col('x31')-pl.col('x39'))**2+(pl.col('x32')-pl.col('x40'))**2).sqrt(),
            _4_45 = ((pl.col('x33')-pl.col('x37'))**2+(pl.col('x34')-pl.col('x38'))**2+(pl.col('x35')-pl.col('x39'))**2+(pl.col('x36')-pl.col('x40'))**2).sqrt(),
        )
        .with_columns(
            ((pl.col(col) - pl.col(col).mean().over('patient_id')) / (pl.col(col).std().over('patient_id') + err)).alias(f'{col}_patient_norm') for col in (num_cols + new_num_cols + new_num_cols2)
        )
        .with_columns(
            count_per_patient = pl.col('isic_id').count().over('patient_id'),
        )
        .with_columns(
            pl.col(cat_cols).cast(pl.Categorical),
        )
        .to_pandas()
        .set_index(id_col)
    )

In [18]:
def preprocess(df_train, df_test):
    global cat_cols
    
    encoder = OneHotEncoder(sparse_output=False, dtype=np.int32, handle_unknown='ignore')
    encoder.fit(df_train[cat_cols])
    
    new_cat_cols = [f'onehot_{i}' for i in range(len(encoder.get_feature_names_out()))]

    df_train[new_cat_cols] = encoder.transform(df_train[cat_cols])
    df_train[new_cat_cols] = df_train[new_cat_cols].astype('category')

    df_test[new_cat_cols] = encoder.transform(df_test[cat_cols])
    df_test[new_cat_cols] = df_test[new_cat_cols].astype('category')

    for col in cat_cols:
        feature_cols.remove(col)

    feature_cols.extend(new_cat_cols)
    cat_cols = new_cat_cols
    
    return df_train, df_test

In [19]:
def custom_metric(estimator, X, y_true):
    y_hat = estimator.predict_proba(X)[:, 1]
    min_tpr = 0.80
    max_fpr = abs(1 - min_tpr)
    
    v_gt = abs(y_true - 1)
    v_pred = np.array([1.0 - x for x in y_hat])
    
    partial_auc_scaled = roc_auc_score(v_gt, v_pred, max_fpr=max_fpr)
    partial_auc = 0.5 * max_fpr**2 + (max_fpr - 0.5 * max_fpr**2) / (1.0 - 0.5) * (partial_auc_scaled - 0.5)
    
    return partial_auc

In [20]:
df_train = read_data(train_path)
df_test = read_data(test_path)
df_subm = pd.read_csv(subm_path, index_col=id_col)

df_train, df_test = preprocess(df_train, df_test)

In [ ]:
#不要な特徴量を削除
least_important_features = ['onehot_32', 'onehot_6', 'onehot_33', 'onehot_30', 'onehot_26', 'onehot_22', 'onehot_36', 'onehot_4']

df_train.drop(columns =least_important_features,inplace = True)
for feature in least_important_features:
    cat_cols.remove(feature)
    feature_cols.remove(feature)

In [ ]:
lgb_params = {
    'objective':        'binary',
    'verbosity':        -1,
    'n_iter':           250,
    'boosting_type':    'gbdt',
    'random_state':     seed,
    'lambda_l1':        0.08758718919397321, 
    'lambda_l2':        0.0039689175176025465, 
    'learning_rate':    0.03231007103195577, 
    'max_depth':        4, 
    'num_leaves':       97, 
    'colsample_bytree': 0.8329551585827726, 
    'colsample_bynode': 0.4025961355653304, 
    'bagging_fraction': 0.7293840594955043, 
    'bagging_freq':     5, 
    'min_data_in_leaf': 85, 
    'scale_pos_weight': 2.7984184778875543,
}

lgb_model = Pipeline([
    ('sampler_1', RandomOverSampler(sampling_strategy= 0.003 , random_state=seed)),
    ('sampler_2', RandomUnderSampler(sampling_strategy=sampling_ratio, random_state=seed)),
    ('classifier', lgb.LGBMClassifier(**lgb_params)),
])

In [ ]:
cb_params = {
    'loss_function':     'Logloss',
    'iterations':        250,
    'verbose':           False,
    'random_state':      seed,
    'max_depth':         6, 
    'learning_rate':     0.07, 
    'scale_pos_weight':  2.6149345838209532, 
    'l2_leaf_reg':       5.9302986748503220, 
    'subsample':         0.8980309133095902, 
    'min_data_in_leaf':  24,
    'cat_features':      cat_cols,
}

cb_model = Pipeline([
    ('sampler_1', RandomOverSampler(sampling_strategy= 0.003 , random_state=seed)),
    ('sampler_2', RandomUnderSampler(sampling_strategy=sampling_ratio, random_state=seed)),
    ('classifier', cb.CatBoostClassifier(**cb_params)),
])

In [ ]:
xgb_params = {
    'enable_categorical': True,
    'tree_method':        'hist',
    'random_state':       seed,
    'learning_rate':      0.08501257473292347, 
    'lambda':             8.879624125465703, 
    'alpha':              0.6779926606782505, 
    'max_depth':          6, 
    'subsample':          0.6012681388711075, 
    'colsample_bytree':   0.7896510578202213, 
    'colsample_bylevel':  0.5476090898823716, 
    'colsample_bynode':   0.9923907895928403,
    'scale_pos_weight':   3.29440313334688,
}

xgb_model = Pipeline([
    ('sampler_1', RandomOverSampler(sampling_strategy= 0.003 , random_state=seed)),
    ('sampler_2', RandomUnderSampler(sampling_strategy=sampling_ratio, random_state=seed)),
    ('classifier', xgb.XGBClassifier(**xgb_params)),
])

In [25]:
estimator = VotingClassifier([
    ('lgb', lgb_model), ('cb', cb_model), ('xgb', xgb_model),
], voting='soft', weights=[0.45,0.35,0.20])

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform
cv = StratifiedGroupKFold(3, shuffle=True, random_state=seed)

# Hyperparameter tuning for LightGBM
param_dist = {
    'classifier__n_estimators': randint(200, 600),
    'classifier__learning_rate': uniform(0.01, 0.09),
    'classifier__max_depth': randint(3, 8),
}
search = RandomizedSearchCV(lgb_model, param_dist, n_iter=15, scoring=custom_metric, cv=cv, random_state=seed, verbose=1)
search.fit(df_train[feature_cols], df_train[target_col], groups=df_train[group_col])
lgb_model = search.best_estimator_

# Hyperparameter tuning for CatBoost
param_dist = {
    'classifier__depth': randint(4, 8),
    'classifier__learning_rate': uniform(0.01, 0.09),
    'classifier__l2_leaf_reg': uniform(1, 7),
}
search = RandomizedSearchCV(cb_model, param_dist, n_iter=15, scoring=custom_metric, cv=cv, random_state=seed, verbose=1)
search.fit(df_train[feature_cols], df_train[target_col], groups=df_train[group_col])
cb_model = search.best_estimator_

# Hyperparameter tuning for XGBoost
param_dist = {
    'classifier__n_estimators': randint(200, 600),
    'classifier__learning_rate': uniform(0.01, 0.09),
    'classifier__max_depth': randint(3, 8),
}
search = RandomizedSearchCV(xgb_model, param_dist, n_iter=15, scoring=custom_metric, cv=cv, random_state=seed, verbose=1)
search.fit(df_train[feature_cols], df_train[target_col], groups=df_train[group_col])
xgb_model = search.best_estimator_


In [ ]:
X = df_train[feature_cols]
y = df_train[target_col]
groups = df_train[group_col]
cv = StratifiedGroupKFold(5, shuffle=True, random_state=seed)

val_score = cross_val_score(
    estimator=estimator, 
    X=X, y=y, 
    cv=cv, 
    groups=groups,
    scoring=custom_metric,
)

np.mean(val_score), val_score

In [28]:
X, y = df_train[feature_cols], df_train[target_col]

estimator.fit(X, y)

/opt/conda/lib/python3.10/site-packages/lightgbm/engine.py:172: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


VotingClassifier(estimators=[('lgb',
                              Pipeline(steps=[('sampler_1',
                                               RandomOverSampler(random_state=42,
                                                                 sampling_strategy=0.003)),
                                              ('sampler_2',
                                               RandomUnderSampler(random_state=42,
                                                                  sampling_strategy=0.01)),
                                              ('classifier',
                                               LGBMClassifier(bagging_fraction=0.7738954452473223,
                                                              bagging_freq=4,
                                                              colsample_bynode=0.4025961355653304,
                                                              colsample_bytree=0.8329551585827726,...
                                                             interaction_constraints=None,
                                                             lambda=8.879624125465703,
                                                             learning_rate=0.08501257473292347,
                                                             max_bin=None,
                                                             max_cat_threshold=None,
                                                             max_cat_to_onehot=None,
                                                             max_delta_step=None,
                                                             max_depth=6,
                                                             max_leaves=None,
                                                             min_child_weight=None,
                                                             missing=nan,
                                                             monotone_constraints=None,
                                                             multi_strategy=None,
                                                             n_estimators=None,
                                                             n_jobs=None, ...))]))],
                 voting='soft', weights=[0.5, 0.4, 0.15])

In [ ]:
df_subm['target'] = estimator.predict_proba(df_test[feature_cols])[:, 1]
df_subm.to_csv('submission_2.csv')

In [30]:
import numpy as np
import pandas as pd
import pandas.api.types
import matplotlib.pyplot as plt
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.model_selection import GroupKFold, StratifiedGroupKFold

import lightgbm as lgb
from scipy.stats import zscore
from numpy import nanmean, nanstd
np.seterr(invalid='ignore')

import copy
import os

In [ ]:
magic_noise_seeds = []
magic_noise_factor = 0.0


In [ ]:
df_train = pd.read_csv("/kaggle/input/isic-2024-challenge/train-metadata.csv")
df_test = pd.read_csv("/kaggle/input/isic-2024-challenge/test-metadata.csv")

max_estimators = 8000    
early_stopping_rounds = 500  
SCORING, QUICK_TEST, SAVE_TRAIN = 1, 2, 3

mode = QUICK_TEST

do_ud = True

load_train_imagepreds_and_cv = True

do_imagenet_uglyduckling = False
import_ud_auged_train = False

oof_imagenet_folds_noise_std = 0.0  

if len(df_test) > 3:
    mode = SCORING

if mode == QUICK_TEST:
    df_train = df_train.head(50000)  
    max_estimators = 500
    do_ud = False  
    
if mode == SAVE_TRAIN:
    max_estimators = 500
    do_ud = True  

main_imagenet_column_name = "imagenet_predict"
image_net_columns = [main_imagenet_column_name]

df_train

In [ ]:
if load_train_imagepreds_and_cv:
    oof_image_net_preds = pd.read_csv("/kaggle/input/isic-2024-imagenet-gen-2-output/oof_predictions.csv")

    oof_image_net_preds = oof_image_net_preds.rename(columns={'oof_prediction': main_imagenet_column_name})

    # df_trainとマージ
    df_train = df_train.merge(oof_image_net_preds[['fold', 'imagenet_predict']], left_index=True, right_index=True, how='left')

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score

def comp_score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str, min_tpr: float=0.80):
    v_gt = abs(np.floor(np.asarray(solution.values))-1)
    v_pred = np.array([1.0 - x for x in submission.values])
    max_fpr = abs(1-min_tpr)
    partial_auc_scaled = roc_auc_score(v_gt, v_pred, max_fpr=max_fpr)
    partial_auc = 0.5 * max_fpr**2 + (max_fpr - 0.5 * max_fpr**2) / (1.0 - 0.5) * (partial_auc_scaled - 0.5)
    return partial_auc

def pauc_score_func(y_true, y_pred):
    y_true = np.asarray(y_true).flatten()
    y_pred = np.asarray(y_pred).flatten()
    y_true_df = pd.DataFrame(y_true, columns=['target'])
    y_pred_df = pd.DataFrame(y_pred, columns=['prediction'])
    return comp_score(y_true_df, y_pred_df, "", min_tpr=0.80)

def process_imagenet_column(df, main_imagenet_column_name, target_column_name, random_seed=42, noise_mean=0, noise_std=0.01):

    np.random.seed(random_seed)
        
    noise = np.random.normal(loc=noise_mean, scale=noise_std, size=len(df))
    df[main_imagenet_column_name] += noise

    df[main_imagenet_column_name] = df[main_imagenet_column_name].clip(0, 1)
    
    return df

print("Before adjust pauc:", pauc_score_func(df_train["target"], df_train["imagenet_predict"]))

target_column_name = 'target'
random_seed = 42 

df_train = process_imagenet_column(df_train, main_imagenet_column_name, target_column_name, random_seed, noise_mean=0, noise_std=0)

print("\nFinal results:")
print(df_train[[main_imagenet_column_name, target_column_name]].head())

print("After adjust pauc:", pauc_score_func(df_train["target"], df_train["imagenet_predict"]))


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from tqdm import tqdm
import h5py
import timm
from torchvision import transforms
from PIL import Image
import io
import albumentations as A
from albumentations.pytorch import ToTensorV2

class ISICDataset(Dataset):
    def __init__(self, hdf5_file, isic_ids, targets=None, transform=None):
        self.hdf5_file = h5py.File(hdf5_file, 'r')
        self.isic_ids = isic_ids
        self.targets = targets
        self.transform = transform

    def __len__(self):
        return len(self.isic_ids)

    def __getitem__(self, idx):
        img_bytes = self.hdf5_file[self.isic_ids[idx]][()]
        img = Image.open(io.BytesIO(img_bytes))
        img = np.array(img)
        
        if self.transform:
            transformed = self.transform(image=img)
            img = transformed['image']
        
        target = self.targets[idx] if self.targets is not None else torch.tensor(-1)
        return img, target

    def __del__(self):
        self.hdf5_file.close()

base_transform = A.Compose([
    A.Resize(224, 224),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

@torch.no_grad()
def ensemble_predict(models, test_loader, device):
    all_predictions = []
    
    for inputs, _ in tqdm(test_loader, desc="Predicting"):
        inputs = inputs.to(device)
        fold_predictions = torch.stack([model(inputs).softmax(dim=1)[:, 1] for model in models])
        avg_predictions = fold_predictions.mean(dim=0)
        all_predictions.append(avg_predictions.cpu())
    
    return torch.cat(all_predictions).numpy()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

df_test = pd.read_csv("/kaggle/input/isic-2024-challenge/test-metadata.csv")
TEST_HDF5_FILE_PATH = '/kaggle/input/isic-2024-challenge/test-image.hdf5'
TRAIN_HDF5_FILE_PATH = '/kaggle/input/isic-2024-challenge/train-image.hdf5'


model_configs = [
    ("/kaggle/input/isic-2024-multifold-v2-offsite-train/v2_model_fold_2_epoch_1.pth", 'tf_efficientnetv2_b1'),
    ("/kaggle/input/isic-2024-multifold-v2-offsite-train/v2_model_fold_4_epoch_1.pth", 'tf_efficientnetv2_b1'),
    ("/kaggle/input/imagenet-143lb-from-isic-2024-imagenet-model-a/best_model.pth", 'tf_efficientnetv2_b1'),
    ("/kaggle/input/isic-2024-effnetv2b0-lb-0-151/effnetv2b0_151lb_isic2024.pth", 'tf_efficientnet_b0')
]

models = [timm.create_model(model_type, pretrained=False, num_classes=2).to(device) for _, model_type in model_configs]
for model, (model_path, _) in zip(models, model_configs):
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

test_dataset = ISICDataset(
    hdf5_file=TEST_HDF5_FILE_PATH,
    isic_ids=df_test['isic_id'].values,
    transform=base_transform,
)
test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False, num_workers=4, pin_memory=True)

predictions = ensemble_predict(models, test_loader, device)
df_test[main_imagenet_column_name] = predictions

print(df_test[main_imagenet_column_name].head())


In [ ]:
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import random

BATCH_SIZE = 128
NUM_WORKERS = 4

os.environ['OMP_NUM_THREADS'] = '1'

def get_infer_model(num_classes=2, model_class = "", model_file = "", is_imagenet = False):
    
    model = timm.create_model(model_class, pretrained=False)
    
    if not is_imagenet:
        in_features = model.classifier.in_features
        model.classifier = torch.nn.Linear(in_features, num_classes)
    
    if torch.cuda.is_available(): 
        model.load_state_dict(torch.load(model_file))
    else:
        model.load_state_dict(torch.load(model_file, map_location=torch.device('cpu')))
        
    model = model.to(device)
    
    return model

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed()


torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(0)

def extract_features(model, loader, device):
    features = []
    isic_ids = []
    model.eval()
    with torch.no_grad():
        for inputs, batch_isic_ids in tqdm(loader, desc="Extracting features"):
            inputs = inputs.to(device)
            output = model(inputs)
            features.append(output.cpu())
            isic_ids.extend(batch_isic_ids)
    return torch.cat(features).numpy(), isic_ids

def calculate_outlier_scores(features):
    print("Normalizing features...")
    scaler = StandardScaler()
    features_normalized = scaler.fit_transform(features)
    
    print("Calculating Isolation Forest scores...")
    iso_forest = IsolationForest(contamination=0.1, random_state=42, n_jobs=1)  
    outlier_scores_if = iso_forest.fit_predict(features_normalized)
    outlier_scores_if = (outlier_scores_if * -1 + 1) / 2
    
    print("Calculating Local Outlier Factor scores...")
    lof = LocalOutlierFactor(n_neighbors=20, contamination=0.1, n_jobs=1)  
    outlier_scores_lof = lof.fit_predict(features_normalized)
    outlier_scores_lof = (outlier_scores_lof * -1 + 1) / 2
    
    print("Combining scores...")
    outlier_scores_combined = (outlier_scores_if + outlier_scores_lof) / 2
    
    return outlier_scores_if, outlier_scores_lof, outlier_scores_combined

def add_outlier_scores_to_df(df, isic_ids, outlier_scores_if, outlier_scores_lof, outlier_scores_combined):
    print("Adding outlier scores to dataframe...")
    temp_df = pd.DataFrame({
        'isic_id': isic_ids,
        'outlier_score_if': outlier_scores_if,
        'outlier_score_lof': outlier_scores_lof,
        'outlier_score_combined': outlier_scores_combined
    })
    return df.merge(temp_df, on='isic_id', how='left')

def gpu_correlation(tensor):
    centered = tensor - tensor.mean(dim=0)
    cov = torch.matmul(centered.t(), centered) / (tensor.size(0) - 1)
    std = torch.sqrt(torch.diag(cov))
    cor = cov / torch.ger(std, std)
    return cor

def plot_correlation_heatmap(df, target_column, feature_columns, device):
    print("Plotting correlation heatmap...")
    tensor = torch.tensor(df[feature_columns + [target_column]].values, dtype=torch.float32).to(device)
    corr = gpu_correlation(tensor)
    corr_cpu = corr.cpu().numpy()
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(corr_cpu, annot=True, cmap='coolwarm', vmin=-1, vmax=1, center=0,
                xticklabels=feature_columns + [target_column],
                yticklabels=feature_columns + [target_column])
    plt.title('Correlation Heatmap: Outlier Scores vs Target')
    plt.show()
    
    return corr_cpu

if do_imagenet_uglyduckling:
    print(f"Using device: {device}")

    print("Loading the model...")
    model = get_infer_model(num_classes=2, model_class="tf_efficientnetv2_b1", model_file="/kaggle/input/isic-2024-tf-efficientnetv2-b1/best_model.pth", is_imagenet=False)

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, 
                              pin_memory=True, worker_init_fn=seed_worker, generator=g)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, 
                             pin_memory=True, worker_init_fn=seed_worker, generator=g)

    print("Extracting features for train set...")
    train_features, train_isic_ids = extract_features(model, train_loader, device)
    print("Extracting features for test set...")
    test_features, test_isic_ids = extract_features(model, test_loader, device)

    print("Calculating outlier scores for train set...")
    train_outlier_scores_if, train_outlier_scores_lof, train_outlier_scores_combined = calculate_outlier_scores(train_features)
    print("Calculating outlier scores for test set...")
    test_outlier_scores_if, test_outlier_scores_lof, test_outlier_scores_combined = calculate_outlier_scores(test_features)

    print("Adding outlier scores to dataframes...")
    df_train = add_outlier_scores_to_df(df_train, train_isic_ids, train_outlier_scores_if, train_outlier_scores_lof, train_outlier_scores_combined)
    df_test = add_outlier_scores_to_df(df_test, test_isic_ids, test_outlier_scores_if, test_outlier_scores_lof, test_outlier_scores_combined)

    outlier_score_columns = ['outlier_score_if', 'outlier_score_lof', 'outlier_score_combined']
    corr_matrix = plot_correlation_heatmap(df_train, 'target', outlier_score_columns, device)

    print("Correlations with target:")
    target_correlations = corr_matrix[-1, :-1]
    for col, corr in zip(outlier_score_columns, target_correlations):
        print(f"{col}: {corr:.4f}")
        
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    del train_features, test_features
    del train_outlier_scores_if, train_outlier_scores_lof, train_outlier_scores_combined
    del test_outlier_scores_if, test_outlier_scores_lof, test_outlier_scores_combined
    del corr_matrix

    import gc
    gc.collect()

    image_net_columns.extend(outlier_score_columns) 
    print(image_net_columns)

In [ ]:
num_cols = [
    'age_approx', 'clin_size_long_diam_mm', 'tbp_lv_A', 'tbp_lv_Aext', 'tbp_lv_B', 'tbp_lv_Bext', 
    'tbp_lv_C', 'tbp_lv_Cext', 'tbp_lv_H', 'tbp_lv_Hext', 'tbp_lv_L', 
    'tbp_lv_Lext', 'tbp_lv_areaMM2', 'tbp_lv_area_perim_ratio', 'tbp_lv_color_std_mean', 
    'tbp_lv_deltaA', 'tbp_lv_deltaB', 'tbp_lv_deltaL', 'tbp_lv_deltaLB',
    'tbp_lv_deltaLBnorm', 'tbp_lv_eccentricity', 'tbp_lv_minorAxisMM',
    'tbp_lv_nevi_confidence', 'tbp_lv_norm_border', 'tbp_lv_norm_color',
    'tbp_lv_perimeterMM', 'tbp_lv_radial_color_std_max', 'tbp_lv_stdL',
    'tbp_lv_stdLExt', 'tbp_lv_symm_2axis', 'tbp_lv_symm_2axis_angle',
    'tbp_lv_x', 'tbp_lv_y', 'tbp_lv_z',
]

num_cols = num_cols + image_net_columns

cat_cols = ["sex", "tbp_tile_type", "tbp_lv_location", "tbp_lv_location_simple"]

def feature_engineering(df):
    df["lesion_size_ratio"] = df["tbp_lv_minorAxisMM"] / df["clin_size_long_diam_mm"]
    df["lesion_shape_index"] = df["tbp_lv_areaMM2"] / (df["tbp_lv_perimeterMM"] ** 2)
    df["hue_contrast"] = (df["tbp_lv_H"] - df["tbp_lv_Hext"]).abs()
    df["luminance_contrast"] = (df["tbp_lv_L"] - df["tbp_lv_Lext"]).abs()
    df["lesion_color_difference"] = np.sqrt(df["tbp_lv_deltaA"] ** 2 + df["tbp_lv_deltaB"] ** 2 + df["tbp_lv_deltaL"] ** 2)
    df["border_complexity"] = df["tbp_lv_norm_border"] + df["tbp_lv_symm_2axis"]
    df["color_uniformity"] = df["tbp_lv_color_std_mean"] / df["tbp_lv_radial_color_std_max"]
    df["3d_position_distance"] = np.sqrt(df["tbp_lv_x"] ** 2 + df["tbp_lv_y"] ** 2 + df["tbp_lv_z"] ** 2) 
    df["perimeter_to_area_ratio"] = df["tbp_lv_perimeterMM"] / df["tbp_lv_areaMM2"]
    df["area_to_perimeter_ratio"] = df["tbp_lv_areaMM2"] / df["tbp_lv_perimeterMM"]
    df["lesion_visibility_score"] = df["tbp_lv_deltaLBnorm"] + df["tbp_lv_norm_color"]
    df["combined_anatomical_site"] = df["anatom_site_general"] + "_" + df["tbp_lv_location"]
    df["symmetry_border_consistency"] = df["tbp_lv_symm_2axis"] * df["tbp_lv_norm_border"]
    df["symmetry_border_consistency2"] = df["tbp_lv_symm_2axis"] * df["tbp_lv_norm_border"] / (df["tbp_lv_symm_2axis"] + df["tbp_lv_norm_border"])
    df["color_consistency"] = df["tbp_lv_stdL"] / df["tbp_lv_Lext"]
    df["color_consistency2"] = df["tbp_lv_stdL"] * df["tbp_lv_Lext"] / (df["tbp_lv_stdL"] + df["tbp_lv_Lext"])
    
    df["size_age_interaction"] = df["clin_size_long_diam_mm"] * df["age_approx"]
    df["hue_color_std_interaction"] = df["tbp_lv_H"] * df["tbp_lv_color_std_mean"]
    df["lesion_severity_index"] = (df["tbp_lv_norm_border"] + df["tbp_lv_norm_color"] + df["tbp_lv_eccentricity"]) / 3
    df["shape_complexity_index"] = df["border_complexity"] + df["lesion_shape_index"]
    df["color_contrast_index"] = df["tbp_lv_deltaA"] + df["tbp_lv_deltaB"] + df["tbp_lv_deltaL"] + df["tbp_lv_deltaLBnorm"]
    df["log_lesion_area"] = np.log(df["tbp_lv_areaMM2"] + 1)
    df["normalized_lesion_size"] = df["clin_size_long_diam_mm"] / df["age_approx"]
    df["mean_hue_difference"] = (df["tbp_lv_H"] + df["tbp_lv_Hext"]) / 2
    df["std_dev_contrast"] = np.sqrt((df["tbp_lv_deltaA"] ** 2 + df["tbp_lv_deltaB"] ** 2 + df["tbp_lv_deltaL"] ** 2) / 3)
    df["color_shape_composite_index"] = (df["tbp_lv_color_std_mean"] + df["tbp_lv_area_perim_ratio"] + df["tbp_lv_symm_2axis"]) / 3
    df["3d_lesion_orientation"] = np.arctan2(df_train["tbp_lv_y"], df_train["tbp_lv_x"])
    df["overall_color_difference"] = (df["tbp_lv_deltaA"] + df["tbp_lv_deltaB"] + df["tbp_lv_deltaL"]) / 3
    df["symmetry_perimeter_interaction"] = df["tbp_lv_symm_2axis"] * df["tbp_lv_perimeterMM"]
    df["comprehensive_lesion_index"] = (df["tbp_lv_area_perim_ratio"] + df["tbp_lv_eccentricity"] + df["tbp_lv_norm_color"] + df["tbp_lv_symm_2axis"]) / 4
    
    df["color_variance_ratio"] = df["tbp_lv_color_std_mean"] / df["tbp_lv_stdLExt"]
    df["border_color_interaction"] = df["tbp_lv_norm_border"] * df["tbp_lv_norm_color"]
    df["size_color_contrast_ratio"] = df["clin_size_long_diam_mm"] / df["tbp_lv_deltaLBnorm"]
    df["age_normalized_nevi_confidence"] = df["tbp_lv_nevi_confidence"] / df["age_approx"]
    df["color_asymmetry_index"] = df["tbp_lv_radial_color_std_max"] * df["tbp_lv_symm_2axis"]
    df["3d_volume_approximation"] = df["tbp_lv_areaMM2"] * np.sqrt(df["tbp_lv_x"]**2 + df["tbp_lv_y"]**2 + df["tbp_lv_z"]**2)
    df["color_range"] = (df["tbp_lv_L"] - df["tbp_lv_Lext"]).abs() + (df["tbp_lv_A"] - df["tbp_lv_Aext"]).abs() + (df["tbp_lv_B"] - df["tbp_lv_Bext"]).abs()
    df["shape_color_consistency"] = df["tbp_lv_eccentricity"] * df["tbp_lv_color_std_mean"]
    df["border_length_ratio"] = df["tbp_lv_perimeterMM"] / (2 * np.pi * np.sqrt(df["tbp_lv_areaMM2"] / np.pi))
    df["age_size_symmetry_index"] = df["age_approx"] * df["clin_size_long_diam_mm"] * df["tbp_lv_symm_2axis"]
    df["age_size_symmetry_index2"] = df["age_approx"] * df["tbp_lv_areaMM2"] * df["tbp_lv_symm_2axis"]

    new_num_cols = [
        "lesion_size_ratio", "lesion_shape_index", "hue_contrast",
        "luminance_contrast", "lesion_color_difference", "border_complexity",
        "color_uniformity", "3d_position_distance", "perimeter_to_area_ratio", "area_to_perimeter_ratio",
        "lesion_visibility_score", "symmetry_border_consistency", "symmetry_border_consistency2", "color_consistency","color_consistency2",

        "size_age_interaction", "hue_color_std_interaction", "lesion_severity_index", 
        "shape_complexity_index", "color_contrast_index", "log_lesion_area",
        "normalized_lesion_size", "mean_hue_difference", "std_dev_contrast",
        "color_shape_composite_index", "3d_lesion_orientation", "overall_color_difference",
        "symmetry_perimeter_interaction", "comprehensive_lesion_index",
        
        "color_variance_ratio", "border_color_interaction", "size_color_contrast_ratio", "age_normalized_nevi_confidence",
        "color_asymmetry_index", "3d_volume_approximation", "color_range", "shape_color_consistency", "border_length_ratio", 
        "age_size_symmetry_index", "age_size_symmetry_index2",
    ]
    new_cat_cols = ["combined_anatomical_site"]
    
    return df, new_num_cols, new_cat_cols

df_train, new_num_cols, new_cat_cols = feature_engineering(df_train.copy())
df_test, _, _ = feature_engineering(df_test.copy())

num_cols = num_cols + new_num_cols
cat_cols = cat_cols + new_cat_cols

In [ ]:
def additional_feature_engineering(df):

    df['asymmetry_ratio'] = df['tbp_lv_symm_2axis'] / df['tbp_lv_perimeterMM']
    df['asymmetry_area_ratio'] = df['tbp_lv_symm_2axis'] / df['tbp_lv_areaMM2']

    df['color_variation_intensity'] = df['tbp_lv_norm_color'] * df['tbp_lv_deltaLBnorm']
    df['color_contrast_ratio'] = df['tbp_lv_deltaLBnorm'] / (df['tbp_lv_L'] + 1e-5)

    df['shape_irregularity'] = df['tbp_lv_perimeterMM'] / (2 * np.sqrt(np.pi * df['tbp_lv_areaMM2']))
    df['border_density'] = df['tbp_lv_norm_border'] / df['tbp_lv_perimeterMM']

    df['size_age_ratio'] = df['clin_size_long_diam_mm'] / (df['age_approx'] + 1e-5)
    df['area_diameter_ratio'] = df['tbp_lv_areaMM2'] / (df['clin_size_long_diam_mm']**2 + 1e-5)

    df['location_size_interaction'] = df.apply(lambda row: f"{row['tbp_lv_location_simple']}_{bin_size(row['clin_size_long_diam_mm'])}", axis=1)
    df['location_age_interaction'] = df.apply(lambda row: f"{row['tbp_lv_location_simple']}_{bin_age(row['age_approx'])}", axis=1)

    df['3d_position_norm'] = np.sqrt(df['tbp_lv_x']**2 + df['tbp_lv_y']**2 + df['tbp_lv_z']**2)
    df['3d_position_angle_xy'] = np.arctan2(df['tbp_lv_y'], df['tbp_lv_x'])
    df['3d_position_angle_xz'] = np.arctan2(df['tbp_lv_z'], df['tbp_lv_x'])

    df['lab_chroma'] = np.sqrt(df['tbp_lv_A']**2 + df['tbp_lv_B']**2)
    df['lab_hue'] = np.arctan2(df['tbp_lv_B'], df['tbp_lv_A'])

    df['texture_contrast'] = df['tbp_lv_stdL'] / (df['tbp_lv_L'] + 1e-5)
    df['texture_uniformity'] = 1 / (1 + df['tbp_lv_color_std_mean'])

    df['color_difference_AB'] = np.sqrt(df['tbp_lv_deltaA']**2 + df['tbp_lv_deltaB']**2)
    df['color_difference_total'] = np.sqrt(df['tbp_lv_deltaA']**2 + df['tbp_lv_deltaB']**2 + df['tbp_lv_deltaL']**2)

    df['anatom_site_encoded'] = df['anatom_site_general'].map(anatom_site_encoding)

    df['sex_encoded'] = df['sex'].map({'male': 0, 'female': 1})

    return df

def bin_size(size):
    if size < 5:
        return 'very_small'
    elif size < 10:
        return 'small'
    elif size < 20:
        return 'medium'
    else:
        return 'large'

def bin_age(age):
    if age < 30:
        return 'young'
    elif age < 60:
        return 'middle_aged'
    else:
        return 'senior'

anatom_site_encoding = {
    'torso': 0,
    'upper extremity': 1,
    'lower extremity': 2,
    'head/neck': 3,
    'palms/soles': 4,
    'oral/genital': 5
}

df_train = additional_feature_engineering(df_train)
df_test = additional_feature_engineering(df_test)

new_num_cols = [
    'asymmetry_ratio', 'asymmetry_area_ratio', 'color_variation_intensity',
    'color_contrast_ratio', 'shape_irregularity', 'border_density',
    'size_age_ratio', 'area_diameter_ratio', '3d_position_norm',
    '3d_position_angle_xy', '3d_position_angle_xz', 'lab_chroma', 'lab_hue',
    'texture_contrast', 'texture_uniformity', 'color_difference_AB',
    'color_difference_total', 'anatom_site_encoded', 'sex_encoded'
]
new_cat_cols = ['location_size_interaction', 'location_age_interaction']

num_cols += new_num_cols
cat_cols += new_cat_cols

In [ ]:
def ugly_duckling_processing(df, num_cols):
    ud_columns = num_cols.copy()
    new_num_cols = []

    include_patient_wide_ud = False  
    
    counter = 0
    
    def calc_ugly_duckling_scores(group, grouping):
        nonlocal counter
        counter += 1
        if counter % 10 == 0: print(".", end="", flush=True)
        z_scores = group[ud_columns].apply(lambda x: zscore(x, nan_policy='omit'))
        ud_scores = np.abs(z_scores)
        prefix = 'ud_' if grouping == 'patient' else 'ud_loc_'
        ud_scores.columns = [f'{prefix}{col}' for col in ud_columns]
        return ud_scores

    print("Analyzing ducklings", end="", flush=True)
    ud_location_col = 'tbp_lv_location'
    ud_scores_loc = df.groupby(['patient_id', ud_location_col])[ud_columns + ['patient_id', ud_location_col]].apply(
        lambda x: calc_ugly_duckling_scores(x, 'location')
    ).reset_index(level=[0, 1], drop=True)
    
    print("\nConcat ducklings")
    df = pd.concat([df, ud_scores_loc], axis=1)
    
    if include_patient_wide_ud:
        print("Analyzing ducklings (part 2)", end="", flush=True)
        ud_scores_patient = df.groupby('patient_id')[ud_columns + ['patient_id']].apply(
            lambda x: calc_ugly_duckling_scores(x, 'patient')
        ).reset_index(level=0, drop=True)
        df = pd.concat([df, ud_scores_patient], axis=1)
        print()  

    print("Extending ducklings")
    new_num_cols.extend([f'ud_loc_{col}' for col in ud_columns])
    if include_patient_wide_ud:
        new_num_cols.extend([f'ud_{col}' for col in ud_columns])

    print("Enhancing ugly duckling features", end="", flush=True)
    
    def calc_percentile_ud_scores(group):
        nonlocal counter
        counter += 1
        if counter % 10 == 0: print(".", end="", flush=True)
        percentiles = group[ud_columns].rank(pct=True)
        return percentiles.add_prefix('ud_percentile_')
    
    counter = 0  
    ud_percentiles = df.groupby('patient_id')[ud_columns].apply(calc_percentile_ud_scores).reset_index(level=0, drop=True)
    df = pd.concat([df, ud_percentiles], axis=1)
    new_num_cols.extend([f'ud_percentile_{col}' for col in ud_columns])
    print() 

    threshold = 2.0  
    if include_patient_wide_ud:
        ud_count = (df[[f'ud_{col}' for col in ud_columns]].abs() > threshold).sum(axis=1)
        df['ud_count_patient'] = ud_count
        new_num_cols.append('ud_count_patient')
    
    ud_count_loc = (df[[f'ud_loc_{col}' for col in ud_columns]].abs() > threshold).sum(axis=1)
    df['ud_count_location'] = ud_count_loc
    new_num_cols.append('ud_count_location')

    if include_patient_wide_ud:
        df['ud_max_severity_patient'] = df[[f'ud_{col}' for col in ud_columns]].abs().max(axis=1)
        new_num_cols.append('ud_max_severity_patient')
    df['ud_max_severity_location'] = df[[f'ud_loc_{col}' for col in ud_columns]].abs().max(axis=1)
    new_num_cols.append('ud_max_severity_location')

    if include_patient_wide_ud:
        df['ud_consistency_patient'] = df[[f'ud_{col}' for col in ud_columns]].abs().std(axis=1)
        new_num_cols.append('ud_consistency_patient')
    df['ud_consistency_location'] = df[[f'ud_loc_{col}' for col in ud_columns]].abs().std(axis=1)
    new_num_cols.append('ud_consistency_location')

    return df, new_num_cols

if do_ud:
    df_train, ud_num_cols = ugly_duckling_processing(df_train.copy(), num_cols)
    df_test, _ = ugly_duckling_processing(df_test.copy(), num_cols)


    num_cols = num_cols + ud_num_cols

In [40]:
train_cols = num_cols + cat_cols

In [41]:
if mode == SAVE_TRAIN:
    df_train.to_csv("isic_2024_train_auged.csv", index=False)

In [ ]:
if import_ud_auged_train:

    train_cols = ['age_approx', 'clin_size_long_diam_mm', 'tbp_lv_A', 'tbp_lv_Aext', 'tbp_lv_B', 'tbp_lv_Bext', 'tbp_lv_C', 'tbp_lv_Cext', 'tbp_lv_H', 'tbp_lv_Hext', 'tbp_lv_L', 'tbp_lv_Lext', 'tbp_lv_areaMM2', 'tbp_lv_area_perim_ratio', 'tbp_lv_color_std_mean', 'tbp_lv_deltaA', 'tbp_lv_deltaB', 'tbp_lv_deltaL', 'tbp_lv_deltaLB', 'tbp_lv_deltaLBnorm', 'tbp_lv_eccentricity', 'tbp_lv_minorAxisMM', 'tbp_lv_nevi_confidence', 'tbp_lv_norm_border', 'tbp_lv_norm_color', 'tbp_lv_perimeterMM', 'tbp_lv_radial_color_std_max', 'tbp_lv_stdL', 'tbp_lv_stdLExt', 'tbp_lv_symm_2axis', 'tbp_lv_symm_2axis_angle', 'tbp_lv_x', 'tbp_lv_y', 'tbp_lv_z', 'lesion_size_ratio', 'lesion_shape_index', 'hue_contrast', 'luminance_contrast', 'lesion_color_difference', 'border_complexity', 'color_uniformity', '3d_position_distance', 'perimeter_to_area_ratio', 'lesion_visibility_score', 'symmetry_border_consistency', 'color_consistency', 'size_age_interaction', 'hue_color_std_interaction', 'lesion_severity_index', 'shape_complexity_index', 'color_contrast_index', 'log_lesion_area', 'normalized_lesion_size', 'mean_hue_difference', 'std_dev_contrast', 'color_shape_composite_index', '3d_lesion_orientation', 'overall_color_difference', 'symmetry_perimeter_interaction', 'comprehensive_lesion_index', 'asymmetry_ratio', 'asymmetry_area_ratio', 'color_variation_intensity', 'color_contrast_ratio', 'shape_irregularity', 'border_density', 'size_age_ratio', 'area_diameter_ratio', '3d_position_norm', '3d_position_angle_xy', '3d_position_angle_xz', 'lab_chroma', 'lab_hue', 'texture_contrast', 'texture_uniformity', 'color_difference_AB', 'color_difference_total', 'anatom_site_encoded', 'sex_encoded', 'ud_loc_age_approx', 'ud_loc_clin_size_long_diam_mm', 'ud_loc_tbp_lv_A', 'ud_loc_tbp_lv_Aext', 'ud_loc_tbp_lv_B', 'ud_loc_tbp_lv_Bext', 'ud_loc_tbp_lv_C', 'ud_loc_tbp_lv_Cext', 'ud_loc_tbp_lv_H', 'ud_loc_tbp_lv_Hext', 'ud_loc_tbp_lv_L', 'ud_loc_tbp_lv_Lext', 'ud_loc_tbp_lv_areaMM2', 'ud_loc_tbp_lv_area_perim_ratio', 'ud_loc_tbp_lv_color_std_mean', 'ud_loc_tbp_lv_deltaA', 'ud_loc_tbp_lv_deltaB', 'ud_loc_tbp_lv_deltaL', 'ud_loc_tbp_lv_deltaLB', 'ud_loc_tbp_lv_deltaLBnorm', 'ud_loc_tbp_lv_eccentricity', 'ud_loc_tbp_lv_minorAxisMM', 'ud_loc_tbp_lv_nevi_confidence', 'ud_loc_tbp_lv_norm_border', 'ud_loc_tbp_lv_norm_color', 'ud_loc_tbp_lv_perimeterMM', 'ud_loc_tbp_lv_radial_color_std_max', 'ud_loc_tbp_lv_stdL', 'ud_loc_tbp_lv_stdLExt', 'ud_loc_tbp_lv_symm_2axis', 'ud_loc_tbp_lv_symm_2axis_angle', 'ud_loc_tbp_lv_x', 'ud_loc_tbp_lv_y', 'ud_loc_tbp_lv_z', 'ud_loc_lesion_size_ratio', 'ud_loc_lesion_shape_index', 'ud_loc_hue_contrast', 'ud_loc_luminance_contrast', 'ud_loc_lesion_color_difference', 'ud_loc_border_complexity', 'ud_loc_color_uniformity', 'ud_loc_3d_position_distance', 'ud_loc_perimeter_to_area_ratio', 'ud_loc_lesion_visibility_score', 'ud_loc_symmetry_border_consistency', 'ud_loc_color_consistency', 'ud_loc_size_age_interaction', 'ud_loc_hue_color_std_interaction', 'ud_loc_lesion_severity_index', 'ud_loc_shape_complexity_index', 'ud_loc_color_contrast_index', 'ud_loc_log_lesion_area', 'ud_loc_normalized_lesion_size', 'ud_loc_mean_hue_difference', 'ud_loc_std_dev_contrast', 'ud_loc_color_shape_composite_index', 'ud_loc_3d_lesion_orientation', 'ud_loc_overall_color_difference', 'ud_loc_symmetry_perimeter_interaction', 'ud_loc_comprehensive_lesion_index', 'ud_loc_asymmetry_ratio', 'ud_loc_asymmetry_area_ratio', 'ud_loc_color_variation_intensity', 'ud_loc_color_contrast_ratio', 'ud_loc_shape_irregularity', 'ud_loc_border_density', 'ud_loc_size_age_ratio', 'ud_loc_area_diameter_ratio', 'ud_loc_3d_position_norm', 'ud_loc_3d_position_angle_xy', 'ud_loc_3d_position_angle_xz', 'ud_loc_lab_chroma', 'ud_loc_lab_hue', 'ud_loc_texture_contrast', 'ud_loc_texture_uniformity', 'ud_loc_color_difference_AB', 'ud_loc_color_difference_total', 'ud_loc_anatom_site_encoded', 'ud_loc_sex_encoded', 'ud_percentile_age_approx', 'ud_percentile_clin_size_long_diam_mm', 'ud_percentile_tbp_lv_A', 'ud_percentile_tbp_lv_Aext', 'ud_percentile_tbp_lv_B', 'ud_percentile_tbp_lv_Bext', 'ud_percentile_tbp_lv_C', 'ud_percentile_tbp_lv_Cext', 'ud_percentile_tbp_lv_H', 'ud_percentile_tbp_lv_Hext', 'ud_percentile_tbp_lv_L', 'ud_percentile_tbp_lv_Lext', 'ud_percentile_tbp_lv_areaMM2', 'ud_percentile_tbp_lv_area_perim_ratio', 'ud_percentile_tbp_lv_color_std_mean', 'ud_percentile_tbp_lv_deltaA', 'ud_percentile_tbp_lv_deltaB', 'ud_percentile_tbp_lv_deltaL', 'ud_percentile_tbp_lv_deltaLB', 'ud_percentile_tbp_lv_deltaLBnorm', 'ud_percentile_tbp_lv_eccentricity', 'ud_percentile_tbp_lv_minorAxisMM', 'ud_percentile_tbp_lv_nevi_confidence', 'ud_percentile_tbp_lv_norm_border', 'ud_percentile_tbp_lv_norm_color', 'ud_percentile_tbp_lv_perimeterMM', 'ud_percentile_tbp_lv_radial_color_std_max', 'ud_percentile_tbp_lv_stdL', 'ud_percentile_tbp_lv_stdLExt', 'ud_percentile_tbp_lv_symm_2axis', 'ud_percentile_tbp_lv_symm_2axis_angle', 'ud_percentile_tbp_lv_x', 'ud_percentile_tbp_lv_y', 'ud_percentile_tbp_lv_z', 'ud_percentile_lesion_size_ratio', 'ud_percentile_lesion_shape_index', 'ud_percentile_hue_contrast', 'ud_percentile_luminance_contrast', 'ud_percentile_lesion_color_difference', 'ud_percentile_border_complexity', 'ud_percentile_color_uniformity', 'ud_percentile_3d_position_distance', 'ud_percentile_perimeter_to_area_ratio', 'ud_percentile_lesion_visibility_score', 'ud_percentile_symmetry_border_consistency', 'ud_percentile_color_consistency', 'ud_percentile_size_age_interaction', 'ud_percentile_hue_color_std_interaction', 'ud_percentile_lesion_severity_index', 'ud_percentile_shape_complexity_index', 'ud_percentile_color_contrast_index', 'ud_percentile_log_lesion_area', 'ud_percentile_normalized_lesion_size', 'ud_percentile_mean_hue_difference', 'ud_percentile_std_dev_contrast', 'ud_percentile_color_shape_composite_index', 'ud_percentile_3d_lesion_orientation', 'ud_percentile_overall_color_difference', 'ud_percentile_symmetry_perimeter_interaction', 'ud_percentile_comprehensive_lesion_index', 'ud_percentile_asymmetry_ratio', 'ud_percentile_asymmetry_area_ratio', 'ud_percentile_color_variation_intensity', 'ud_percentile_color_contrast_ratio', 'ud_percentile_shape_irregularity', 'ud_percentile_border_density', 'ud_percentile_size_age_ratio', 'ud_percentile_area_diameter_ratio', 'ud_percentile_3d_position_norm', 'ud_percentile_3d_position_angle_xy', 'ud_percentile_3d_position_angle_xz', 'ud_percentile_lab_chroma', 'ud_percentile_lab_hue', 'ud_percentile_texture_contrast', 'ud_percentile_texture_uniformity', 'ud_percentile_color_difference_AB', 'ud_percentile_color_difference_total', 'ud_percentile_anatom_site_encoded', 'ud_percentile_sex_encoded', 'ud_count_location', 'ud_max_severity_location', 'ud_consistency_location', 'sex', 'tbp_tile_type', 'tbp_lv_location', 'tbp_lv_location_simple', 'combined_anatomical_site', 'location_size_interaction', 'location_age_interaction']
    cat_cols = ['sex', 'tbp_tile_type', 'tbp_lv_location', 'tbp_lv_location_simple', 'combined_anatomical_site', 'location_size_interaction', 'location_age_interaction']

    df_train = pd.read_csv("/kaggle/input/isic-2024-tabular-feature-generation/isic_2024_train_auged.csv")

    missing_cols = set(df_train.columns) - set(df_test.columns)
    new_cols = pd.DataFrame({col: df_train[col].iloc[0] for col in missing_cols}, index=df_test.index).astype(df_train[list(missing_cols)].dtypes)
    df_test = pd.concat([df_test, new_cols], axis=1)[df_train.columns]

In [ ]:
category_encoder = OrdinalEncoder(
    categories='auto',
    dtype=int,
    handle_unknown='use_encoded_value',
    unknown_value=-2,
    encoded_missing_value=-1,
)

X_cat = category_encoder.fit_transform(df_train[cat_cols])
for c, cat_col in enumerate(cat_cols):
    df_train[cat_col] = X_cat[:, c]

X_cat = category_encoder.fit_transform(df_test[cat_cols])
for c, cat_col in enumerate(cat_cols):
    df_test[cat_col] = X_cat[:, c]

In [ ]:

base_removals_1 = [
    "ud_percentile_log_lesion_area", "ud_count_location", "texture_uniformity", "ud_loc_tbp_lv_y",
    "log_lesion_area", "ud_percentile_symmetry_border_consistency", "ud_percentile_3d_position_distance", "ud_loc_color_contrast_ratio",
    "ud_loc_3d_position_angle_xy", "tbp_lv_perimeterMM", "normalized_lesion_size", "ud_loc_area_diameter_ratio",
    "ud_consistency_location", "tbp_lv_deltaA", "tbp_lv_C", "ud_percentile_tbp_lv_norm_color",
    "ud_loc_std_dev_contrast", "ud_loc_texture_uniformity", "ud_loc_tbp_lv_radial_color_std_max", "perimeter_to_area_ratio",
    "lab_hue", "ud_loc_color_consistency", "tbp_lv_B", "ud_percentile_age_approx",
    "ud_loc_3d_position_distance", "ud_percentile_lesion_shape_index", "ud_loc_tbp_lv_area_perim_ratio", "tbp_lv_minorAxisMM",
    "ud_loc_tbp_lv_deltaB", "ud_percentile_color_contrast_index", "ud_loc_overall_color_difference", "shape_complexity_index",
    "ud_loc_border_complexity", "sex_encoded", "mean_hue_difference", "ud_loc_lesion_visibility_score",
    "tbp_lv_A", "tbp_lv_Bext", "ud_loc_tbp_lv_stdL", "ud_loc_asymmetry_ratio",
    "ud_loc_tbp_lv_deltaA"
    ]



train_cols = [col for col in train_cols if col not in base_removals_1]


base_removals_2 = ["ud_loc_tbp_lv_eccentricity", "ud_percentile_size_age_interaction", "ud_percentile_tbp_lv_C", "ud_percentile_lesion_visibility_score", "ud_percentile_tbp_lv_L",
                   "ud_loc_tbp_lv_symm_2axis_angle", "asymmetry_area_ratio", "ud_loc_mean_hue_difference", "tbp_lv_H", "ud_percentile_tbp_lv_Lext", "ud_percentile_tbp_lv_deltaL",
                   "ud_percentile_size_age_ratio", "comprehensive_lesion_index", "tbp_lv_deltaB", "ud_loc_symmetry_border_consistency", "ud_percentile_tbp_lv_perimeterMM", "hue_color_std_interaction"]


train_cols = [col for col in train_cols if col not in base_removals_2]


base_removals_3 = ["tbp_lv_Hext", "ud_percentile_color_difference_AB", "hue_contrast", "ud_loc_color_difference_total"]


train_cols = [col for col in train_cols if col not in base_removals_3]

In [45]:
if not load_train_imagepreds_and_cv:
    gkf = GroupKFold(n_splits=5)

    df_train["fold"] = -1
    for idx, (train_idx, val_idx) in enumerate(gkf.split(df_train, df_train["target"], groups=df_train["patient_id"])):
        df_train.loc[val_idx, "fold"] = idx

In [46]:
df_train['robustness_score'] = 100

df_train.loc[df_train['lesion_id'].notna(), 'robustness_score'] = 200

df_train.loc[df_train['iddx_1'] == "Indeterminate", 'robustness_score'] = 50

df_train['robustness_score'] += 0.25 * df_train['tbp_lv_dnn_lesion_confidence'].fillna(0)

def create_sample_weights(df, robustness_column='robustness_score', min_weight=1, max_weight=10):
    min_score = df[robustness_column].min()
    max_score = df[robustness_column].max()
    weights = min_weight + (max_weight - min_weight) * (df[robustness_column] - min_score) / (max_score - min_score)
    return weights

In [ ]:
importances = np.mean([model.feature_importance(importance_type='gain') for model in models], axis=0)
df_imp = pd.DataFrame({
    "feature": models[0].feature_name(),  
    "importance": importances
}).sort_values("importance", ascending=False).reset_index(drop=True)

In [ ]:
model_directory="saved_models"

def save_models(models)
    
    if not os.path.exists(model_directory):
        os.makedirs(model_directory)
    
    for i, model in enumerate(models):
        model_path = os.path.join(model_directory, f"model_fold_{i}.txt")
        model.save_model(model_path)
    
    print(f"Models saved in {model_directory}")

def load_models(directory="saved_models"):
    models = []
    for i in range(5):  
        model_path = os.path.join(directory, f"model_fold_{i}.txt")
        if os.path.exists(model_path):
            model = lgb.Booster(model_file=model_path)
            models.append(model)
    
    print(f"Loaded {len(models)} models from {directory}")
    return models

save_models(models)

Models saved in saved_models


In [ ]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def get_probabilities(model, X):
    if hasattr(model, 'predict_proba'):
        return model.predict_proba(X)[:, 1]
    else:
        return sigmoid(model.predict(X))

def stacking_ensemble_lgb_new(models, X_train, y_train, X_test, train_cols, meta_model=LogisticRegression()):
    num_models = len(models)
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    oof_preds = np.zeros((len(y_train), num_models))
    test_preds = np.zeros((len(X_test), num_models))

    for i, model in enumerate(models):

        for train_idx, val_idx in kf.split(X_train):
            oof_preds[val_idx, i] = get_probabilities(model, X_train.iloc[val_idx][train_cols])
        
        test_preds[:, i] = get_probabilities(model, X_test[train_cols])

    meta_model.fit(oof_preds, y_train)
    final_preds = meta_model.predict_proba(test_preds)[:, 1]
    return final_preds

In [ ]:
def generate_seeded_noise(seed, size, noise_factor):
    return np.zeros(size)

def apply_stacked_noise(preds, magic_noise_seeds, noise_factor=0.025):
    return preds


In [ ]:
preds = stacking_ensemble_lgb_new(models, df_train, df_train["target"], df_test, train_cols)

In [ ]:
class EarlyStoppingException(Exception):
    pass

class EarlyStoppingByPAUC:
    def __init__(self, stopping_cycles, period=1, eval_set=None):
        self.stopping_cycles = stopping_cycles
        self.period = period
        self.eval_set = eval_set
        self.best_score = -np.inf
        self.best_iteration = 0
        self.best_model = None
        self.counter = 0

    def __call__(self, env):
        if self.eval_set is None or env.iteration % self.period != 0:
            return False

        y_true = self.eval_set[0][1]
        y_pred = env.model.predict(self.eval_set[0][0], num_iteration=env.iteration)
        current_score = pauc_score_func(y_true, y_pred)

        print(f"Iteration {env.iteration}, Current pAUC: {current_score:.5f}")

        if current_score > self.best_score:
            self.best_score = current_score
            self.best_iteration = env.iteration
            self.best_model_state = copy.deepcopy(env.model)
            self.counter = 0
        else:
            self.counter += 1

        if self.counter >= self.stopping_cycles:
            print(f"Early stopping at iteration {env.iteration}")
            print(f"Best iteration: {self.best_iteration}")
            print(f"Best pAUC: {self.best_score:.5f}")
            raise EarlyStoppingException()
            return True

        return False

    
def train_lgbm_with_early_stopping(df_train, train_cols, use_early_stopping=True, early_stopping_rounds=100):
    lgb_params = {
        'objective': 'binary',
        "random_state": 42,
        "n_estimators": max_estimators,
        'learning_rate': 0.001,
        'num_leaves': 37,
        'min_data_in_leaf': 57,
        'bagging_freq': 1,
        'pos_bagging_fraction': 0.74,
        'neg_bagging_fraction': 0.07,
        'feature_fraction': 0.57,
        'lambda_l1': 0.21,
        'lambda_l2': 0.7,
        "verbosity": -1
    }
    scores = []
    models = []
    for fold in range(5):
        df_train_fold = df_train[df_train["fold"] != fold].reset_index(drop=True)
        df_valid_fold = df_train[df_train["fold"] == fold].reset_index(drop=True)
        
        train_weights = create_sample_weights(df_train_fold)
        
        train_dataset = lgb.Dataset(df_train_fold[train_cols], df_train_fold["target"], weight=train_weights)
        
        valid_dataset = lgb.Dataset(df_valid_fold[train_cols], df_valid_fold["target"], reference=train_dataset)
        
        eval_set = [(df_valid_fold[train_cols], df_valid_fold["target"])]
        callbacks = []
        
        if use_early_stopping:
            rounds_between_pauc_check = 50
            early_stopping_callback = EarlyStoppingByPAUC(
                stopping_cycles=early_stopping_rounds // rounds_between_pauc_check,
                period=rounds_between_pauc_check,
                eval_set=eval_set
            )
            callbacks.append(early_stopping_callback)
        
        try:
            model = lgb.train(
                lgb_params,
                train_dataset,
                valid_sets=[valid_dataset],
                callbacks=callbacks,
                num_boost_round=lgb_params['n_estimators']
            )

        except EarlyStoppingException:
            print(f"Training stopped early.")
        else:
            print(f"Completed without early stopping")

        model = early_stopping_callback.best_model_state
        score = early_stopping_callback.best_score
        print(f"Using best model from iteration {early_stopping_callback.best_iteration}")
        print(f"Fold {fold} / Partial AUC Score: {score:.5f}")            
        
        scores.append(score)        
        models.append(model)
    
    print("\nAverage pAUC:", np.mean(scores))
    return models, scores

# Usage
use_early_stopping = True
models, scores = train_lgbm_with_early_stopping(df_train, train_cols, use_early_stopping, early_stopping_rounds)
np.mean(scores)

In [ ]:
df_sub = pd.read_csv("/kaggle/input/isic-2024-challenge/sample_submission.csv")
df_sub["target"] = preds
df_sub.to_csv("submission_3.csv", index=False)

In [ ]:
sm1 = pd.read_csv('submission_1.csv')
sm2 = pd.read_csv('submission_2.csv')
sm3 = pd.read_csv('submission_3.csv')

In [ ]:
sms = pd.merge(sm1,sm2, on=['isic_id'])
sms = pd.merge(sms,sm3, on=['isic_id'])
sms['target'] = (sms['target_x']/100000 + sms['target_y']/1000 + sms['target']) / 3
sub = sms[['isic_id','target']]
sub.to_csv('submission.csv', index=False, float_format='%.7f')